In [ ]:
# metrics_long: LONG TREND FOLLOWING STRATEGY [if the condition for opening a new position is not met, consider whether the conditions for stopping previous PnL are met] & PERFORMANCE METRICS PER TRADE (May-13)
# metrics_short: SHORT TREND FOLLOWING STRATEGY [if the condition for opening a new position is not met, consider whether the conditions for stopping pervious PnL are met] & PERFORMANCE METRICS PER TRADE (May-14)
# ergodicity_optimize_long: FOR LONG TREND FOLLOWING STRATEGY - OPTIMIZE (ADX, BIAS) PAIRS BASED ON GAIN-TO-PAIN RATIO, FINAL PROFITS (May-17)
# ergodicity_optimize_short: FOR SHORT TREND FOLLOWING STRATEGY - OPTIMIZE (ADX, BIAS) PAIRS BASED ON GAIN-TO-PAIN RATIO, FINAL PROFITS (May-17)
# CTA_ver0_long: IMPLEMENT LONG TREND FOLLOWING STRATEGY WITH OPTIMAL (ADX, BIAS) PAIR FOR A SINGLE CONTRACT TO GENERATE TRADE RECORDS, DAILY PNLS, AND PERFORMANCE METRICS (May-13)
# CTA_ver0_short: IMPLEMENT SHORT TREND FOLLOWING STRATEGY WITH OPTIMAL (ADX, BIAS) PAIR FOR A SINGLE CONTRACT TO GENERATE TRADE RECORDS, DAILY PNLS, AND PERFORMANCE METRICS (May-14)
# CTA_ver1: IMPLEMENT UNI-DIRACTIONAL CTA STRATEGY WITH OPTIMAL (ADX, BIAS) PAIRS FOR LISTED CONTRACTS TO GENERATE TRADE RECORDS, DAILY PNLS, AND PERFORMANCE METRICS
# CTA_ver2: IMPLEMENT CTA STRATEGY WITH OPTIMAL (ADX, BIAS) PAIRS FOR ALL CONTRACTS TO GENERATE TRADE RECORDS, DAILY PNLS, AND PERFORMANCE METRICS (May-20)
# CTA_ver3: IMPLEMENT CTA STRATEGY FOR ALL CONTRACTS IN MONETARY BASIS (THEORETICALLY ENOUGH INITIAL CAPITAL)

In [ ]:
# metrics_long: LONG TREND FOLLOWING STRATEGY [if the condition for opening a new position is not met, consider whether the conditions for stopping previous PnL are met] & PERFORMANCE METRICS PER TRADE (May-13)
# coding="utf-8"
import os
import sys
sys.path.insert(0, os.getcwd())
import talib as ta
import pandas as pd
import numpy as np
import datetime

def process_csv(file_path):
    # LOAD DATA AND DECIDE TRADEABILITY
    df = pd.read_csv(file_path, engine='python')
    df.columns = df.columns.str.replace(' ', '')
    df.fillna(value={'open': df['close']}, inplace=True)
    df.interpolate(method='nearest', inplace=True)
    df = df.dropna(axis=1, how='all')
    df = df.iloc[::-1]
    df = df.reset_index(drop=True)
    df['trading_date'] = pd.to_datetime(df['trading_date'], format='%m/%d/%Y')
    df['contract_year'] = df['contract_year'].apply(lambda x: int('20' + x.split('^')[1] + x.split('^')[0]))
    if 'N' in df['contract_month'].values:
        month_code = 'N'
    elif 'V' in df['contract_month'].values:
        month_code = 'V'
    elif 'X' in df['contract_month'].values:
        month_code = 'X'
    elif 'Z' in df['contract_month'].values:
        month_code = 'Z'
    else:
        month_code = input("Enter the month code of the contract (N, V, X, Z): ")
    expire_dates = []
    for i in range(len(df)):
        contract_year = df['contract_year'][i]
        if month_code == 'N':
            expire_date = pd.Timestamp(str(contract_year) + '-07-01')
        elif month_code == 'V':
            expire_date = pd.Timestamp(str(contract_year) + '-10-01')
        elif month_code == 'X':
            expire_date = pd.Timestamp(str(contract_year) + '-11-01')
        elif month_code == 'Z':
            expire_date = pd.Timestamp(str(contract_year) + '-12-01')
        else:
            print("Invalid month code!")
            break
        expire_dates.append(expire_date)
        if df['trading_date'][i].year == contract_year:
            if df['trading_date'][i] > expire_date:
                df.loc[i, 'tradeability'] = 'nontradeable'
            else:
                df.loc[i, 'tradeability'] = 'tradeable'
        elif df['trading_date'][i].year != contract_year and df['trading_date'][i].month < expire_date.month:
            df.loc[i, 'tradeability'] = 'not to trade'
        else:
            df.loc[i, 'tradeability'] = 'tradeable'
    df['expire_date'] = expire_dates
    df['EMA20'] = ta.EMA(df['close'], timeperiod=20)
    df['EMA10'] = ta.EMA(df['close'], timeperiod=10)
    df['EMA5'] = ta.EMA(df['close'], timeperiod=5)
    df['ADX'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    df['BIAS'] = (df['close'] - df['close'].rolling(window=20).mean()) / df['close'].rolling(window=20).mean() * 100
    position = 0  
    initial_cash = 10000
    current_cash = initial_cash
    current_capital = initial_cash
    total_trading_cost = 0
    trades = []
    daily = []
    # LONG TREND FOLLWOING STRATEGY [CONSIDER STOP P/L IF CONTINOUS OPENING CONDITION IS NOT MET]
    # USE (ADX, BIAS) = (35, 10) FOR CTZ AS AN EXAMPLE
    for i in range(1, len(df)):
        trading_date = df['trading_date'][i]
        if df['tradeability'][i] == 'tradeable':
            if df['ADX'][i-1] > 35 and \
               df['close'][i-1] > df['EMA10'][i-1] and \
               df['EMA10'][i-1] > df['EMA20'][i-1] and \
               df['EMA10'][i-1] > df['EMA10'][i-2] and \
               df['EMA20'][i-1] > df['EMA20'][i-2] and \
               df['BIAS'][i-1] < 10 and \
               position < 3:
                action = 'BUY'
                price = df['open'][i]
                position += 1
                trading_cost = 0.0005 * price 
                total_trading_cost += trading_cost
                current_cash -= (price + trading_cost)
                current_capital = current_cash + (position * df['open'][i]) 
                trades.append([trading_date, action, price, position, trading_cost, current_capital]) 
            elif position > 0:
                if df['close'][i-1] < df['EMA10'][i-1] or \
                   df['BIAS'][i-1] > 10:
                    action = 'SELL'
                    price = df['open'][i]
                    current_cash += price * position
                    trading_cost = 0.0005 * price * position
                    position = 0
                    total_trading_cost += trading_cost
                    current_cash -= trading_cost
                    current_capital = current_cash
                    trades.append([trading_date, action, price, position, trading_cost, current_capital]) 
                else:
                    action = 'HOLD'
                    price = None
                    trading_cost = 0
                    if df['close'][i] > df['close'][i-1] or \
                       df['close'][i] == df['close'][i-1]:  
                        current_cash += position * (df['close'][i] - df['close'][i-1])
                        current_capital += position * (df['close'][i] - df['close'][i-1])
                    else:
                        current_cash -= position * (df['close'][i-1] - df['close'][i])	
                        current_capital -= position * (df['close'][i-1] - df['close'][i])
                    #trades.append([trading_date, action, price, position, trading_cost, current_capital]) 
            else:
                action = None
                #price = None
                trading_cost = 0
                #current_cash = current_capital
        else:
            action = None
            #price = None
            trading_cost = 0
            #current_cash = current_cash   
    trades_df = pd.DataFrame(trades, columns=['date', 'action', 'price', 'current_share_holding', 'trading_cost', 'current_capital'])
    # CALCULATE PERFORMANCE METRICS
    trades_df['daily_returns'] = trades_df['current_capital'].pct_change()
    final_sharpe_ratio = trades_df['daily_returns'].mean() / trades_df['daily_returns'].std()
    final_max_drawdown = (1 - (trades_df['current_capital'] / trades_df['current_capital'].cummax())).max() # max_dd = MAX(1 - current_capital / high_water_mark)
    daily_returns = trades_df['daily_returns']
    def calculate_gain_to_pain_ratio(daily_returns): #https://jackschwager.com/market-wizards-search-part-2-the-performance-statistics-i-use/
        total_pain = sum([r if r < 0 else 0 for r in daily_returns])
        total_gain = sum([r if r > 0 else 0 for r in daily_returns])
        if total_pain == 0:
            return float('inf')  # return infinity if total pain is zero to avoid division by zero
        else:
            return abs(total_gain / total_pain)
    final_gain_to_pain_ratio = calculate_gain_to_pain_ratio(daily_returns)
    #final_pain_to_gain_ratio = (-final_max_drawdown) / trades_df['current_capital'].cummax().max()
    trades_df['final_sharpe_ratio'] = final_sharpe_ratio
    trades_df['final_max_drawdown'] = final_max_drawdown
    trades_df['final_gain_to_pain_ratio'] = final_gain_to_pain_ratio
    # SAVE RESULTS
    result_folder = "result"
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)
    trades_excel_path = os.path.join(result_folder, os.path.basename(file_path).replace(".csv", "_trades.xlsx"))
    trades_df.to_excel(trades_excel_path, index=False, float_format='%.4f')
    #trades_df.to_csv(os.path.join(result_folder, os.path.basename(file_path).replace(".csv", "_trades.csv")), index=False)
    print(f"Processed {file_path} and saved trade record to {result_folder}")
    print("Final Sharpe Ratio:", final_sharpe_ratio)
    print("Final Max Drawdown:", final_max_drawdown)
    print("Final Gain-to-Pain Ratio:", final_gain_to_pain_ratio)
    print("Total Trading Cost:", total_trading_cost)
    # OUTPUT METRICS TO "metrics.csv"
    metrics_df = pd.DataFrame({
        'Code': [os.path.basename(file_path).replace(".csv", "")],
        'Final_Sharpe_Ratio': [final_sharpe_ratio],
        'Final_Max_Drawdown': [final_max_drawdown],
        'Final_Gain_to_Pain_Ratio': [final_gain_to_pain_ratio]
    })
    metrics_file = os.path.join(output_folder, "metrics.csv")
    if not os.path.exists(metrics_file):
        metrics_df.to_csv(metrics_file, index=False)
    else:
        metrics_df.to_csv(metrics_file, mode='a', header=False, index=False)
    print(f"Saved metrics to {metrics_file}")
if __name__ == "__main__":
    input_folder = "C:/Users/Jiaxu/Desktop/quant/test"
    output_folder = "C:/Users/Jiaxu/Desktop/quant/result"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_folder, filename)
            process_csv(file_path)

In [ ]:
# metrics_short: SHORT TREND FOLLOWING STRATEGY [if the condition for opening a new position is not met, consider whether the conditions for stopping pervious PnL are met] & PERFORMANCE METRICS PER TRADE (May-14)
# coding="utf-8"
import os
import sys
sys.path.insert(0, os.getcwd())
import talib as ta
import pandas as pd
import numpy as np
import datetime

def process_csv(file_path):
    # LOAD DATA AND DECIDE TRADEABILITY
    df = pd.read_csv(file_path, engine='python')
    df.columns = df.columns.str.replace(' ', '')
    df.fillna(value={'open': df['close']}, inplace=True)
    df.interpolate(method='nearest', inplace=True)
    df = df.dropna(axis=1, how='all')
    df = df.iloc[::-1]
    df = df.reset_index(drop=True)
    df['trading_date'] = pd.to_datetime(df['trading_date'], format='%m/%d/%Y')
    df['contract_year'] = df['contract_year'].apply(lambda x: int('20' + x.split('^')[1] + x.split('^')[0]))
    if 'N' in df['contract_month'].values:
        month_code = 'N'
    elif 'V' in df['contract_month'].values:
        month_code = 'V'
    elif 'X' in df['contract_month'].values:
        month_code = 'X'
    elif 'Z' in df['contract_month'].values:
        month_code = 'Z'
    else:
        month_code = input("Enter the month code of the contract (N, V, X, Z): ")
    expire_dates = []
    for i in range(len(df)):
        contract_year = df['contract_year'][i]
        if month_code == 'N':
            expire_date = pd.Timestamp(str(contract_year) + '-07-01')
        elif month_code == 'V':
            expire_date = pd.Timestamp(str(contract_year) + '-10-01')
        elif month_code == 'X':
            expire_date = pd.Timestamp(str(contract_year) + '-11-01')
        elif month_code == 'Z':
            expire_date = pd.Timestamp(str(contract_year) + '-12-01')
        else:
            print("Invalid month code!")
            break
        expire_dates.append(expire_date)
        if df['trading_date'][i].year == contract_year:
            if df['trading_date'][i] > expire_date:
                df.loc[i, 'tradeability'] = 'nontradeable'
            else:
                df.loc[i, 'tradeability'] = 'tradeable'
        elif df['trading_date'][i].year != contract_year and df['trading_date'][i].month < expire_date.month:
            df.loc[i, 'tradeability'] = 'not to trade'
        else:
            df.loc[i, 'tradeability'] = 'tradeable'
    df['expire_date'] = expire_dates
    df['EMA20'] = ta.EMA(df['close'], timeperiod=20)
    df['EMA10'] = ta.EMA(df['close'], timeperiod=10)
    df['EMA5'] = ta.EMA(df['close'], timeperiod=5)
    df['ADX'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    df['BIAS'] = (df['close'] - df['close'].rolling(window=20).mean()) / df['close'].rolling(window=20).mean() * 100
    position = 0  
    initial_cash = 10000
    current_cash = initial_cash
    current_capital = initial_cash
    total_trading_cost = 0
    trades = []
    # SHORT TREND FOLLWOING STRATEGY [CONSIDER STOP P/L IF CONTINOUS OPENING CONDITION IS NOT MET]
    # USE (ADX, BIAS) = (30, -8) FOR KCN AS AN EXAMPLE
    for i in range(1, len(df)):
        trading_date = df['trading_date'][i]
        if df['tradeability'][i] == 'tradeable':
            if df['ADX'][i-1] > 30 and \
               df['close'][i-1] < df['EMA10'][i-1] and \
               df['EMA10'][i-1] < df['EMA20'][i-1] and \
               df['EMA10'][i-1] < df['EMA10'][i-2] and \
               df['EMA20'][i-1] < df['EMA20'][i-2] and \
               df['BIAS'][i-1] > -8 and \
               position < 1:
                action = 'SELL'
                price = df['open'][i]
                position += 1
                trading_cost = 0.0005 * price 
                total_trading_cost += trading_cost
                current_cash -= (price + trading_cost)
                current_capital = current_cash + (position * df['open'][i])
                trades.append([trading_date, action, price, position, trading_cost, current_capital]) 
            elif position > 0:
                if df['close'][i-1] > df['EMA10'][i-1] or \
                   df['BIAS'][i-1] < -8:
                    action = 'BUY'
                    price = df['open'][i]
                    current_cash += price * position
                    trading_cost = 0.0005 * price * position
                    position = 0
                    total_trading_cost += trading_cost
                    current_cash -= trading_cost
                    current_capital = current_cash 
                    trades.append([trading_date, action, price, position, trading_cost, current_capital]) 
                else:
                    action = 'HOLD'
                    price = None
                    trading_cost = 0
                    if df['close'][i] < df['close'][i-1] or \
                       df['close'][i] == df['close'][i-1]:
                        current_cash += position * (df['close'][i-1] - df['close'][i])
                        current_capital += position * (df['close'][i-1] - df['close'][i])
                    else:
                        current_cash -= position * (df['close'][i] - df['close'][i-1])
                        current_capital -= position * (df['close'][i] - df['close'][i-1])
                    #trades.append([trading_date, action, price, position, trading_cost, current_capital]) 
            else:
                action = None
                #price = None
                trading_cost = 0
                #current_cash = current_cash
        else:
            action = None
            #price = None
            trading_cost = 0
            #current_cash = current_cash 
    trades_df = pd.DataFrame(trades, columns=['trading_date', 'action', 'price', 'current_share_holding', 'trading_cost', 'current_capital'])
    # CALCULATE PERFORMANCE METRICS
    trades_df['daily_returns'] = trades_df['current_capital'].pct_change()
    final_sharpe_ratio = trades_df['daily_returns'].mean() / trades_df['daily_returns'].std()
    final_max_drawdown = (1 - (trades_df['current_capital'] / trades_df['current_capital'].cummax())).max() # max_dd = MAX(1 - current_capital / high_water_mark)
    daily_returns = trades_df['daily_returns']
    def calculate_gain_to_pain_ratio(daily_returns): #https://jackschwager.com/market-wizards-search-part-2-the-performance-statistics-i-use/
        total_pain = sum([r if r < 0 else 0 for r in daily_returns])
        total_gain = sum([r if r > 0 else 0 for r in daily_returns])
        if total_pain == 0:
            return float('inf')  # return infinity if total pain is zero to avoid division by zero
        else:
            return abs(total_gain / total_pain)
    final_gain_to_pain_ratio = calculate_gain_to_pain_ratio(daily_returns)
    #final_pain_to_gain_ratio = (-final_max_drawdown) / trades_df['current_capital'].cummax().max()
    trades_df['final_sharpe_ratio'] = final_sharpe_ratio
    trades_df['final_max_drawdown'] = final_max_drawdown
    trades_df['final_gain_to_pain_ratio'] = final_gain_to_pain_ratio
    # SAVE RESULTS
    result_folder = "result"
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)
    trades_excel_path = os.path.join(result_folder, os.path.basename(file_path).replace(".csv", "_trades.xlsx"))
    trades_df.to_excel(trades_excel_path, index=False, float_format='%.4f')
    #trades_df.to_csv(os.path.join(result_folder, os.path.basename(file_path).replace(".csv", "_trades.csv")), index=False)
    print(f"Processed {file_path} and saved trade record to {result_folder}")
    print("Final Sharpe Ratio:", final_sharpe_ratio)
    print("Final Max Drawdown:", final_max_drawdown)
    print("Final Gain-to-Pain Ratio:", final_gain_to_pain_ratio)
    print("Total Trading Cost:", total_trading_cost)
    # OUTPUT METRICS TO "metrics.csv"
    metrics_df = pd.DataFrame({
        'Code': [os.path.basename(file_path).replace(".csv", "")],
        'Final_Sharpe_Ratio': [final_sharpe_ratio],
        'Final_Max_Drawdown': [final_max_drawdown],
        'Final_Gain_to_Pain_Ratio': [final_gain_to_pain_ratio]
    })
    metrics_file = os.path.join(output_folder, "metrics.csv")
    if not os.path.exists(metrics_file):
        metrics_df.to_csv(metrics_file, index=False)
    else:
        metrics_df.to_csv(metrics_file, mode='a', header=False, index=False)
    print(f"Saved metrics to {metrics_file}")
if __name__ == "__main__":
    input_folder = "C:/Users/Jiaxu/Desktop/quant/test"
    output_folder = "C:/Users/Jiaxu/Desktop/quant/result"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_folder, filename)
            process_csv(file_path)

In [ ]:
# ergodicity_optimize_long: FOR LONG TREND FOLLOWING STRATEGY - OPTIMIZE (ADX, BIAS) PAIRS BASED ON GAIN-TO-PAIN RATIO, FINAL PROFITS (May-17)
# coding="utf-8"
import os
import sys
sys.path.insert(0, os.getcwd())
import talib as ta
import pandas as pd
import numpy as np
import datetime

def process_csv(file_path, output_folder):
    # LOAD DATA AND DECIDE TRADEABILITY
    df = pd.read_csv(file_path, engine='python')
    df.columns = df.columns.str.replace(' ', '')
    df.fillna(value={'open': df['close']}, inplace=True)
    df.interpolate(method='nearest', inplace=True)
    df = df.dropna(axis=1, how='all')
    df = df.iloc[::-1]
    df = df.reset_index(drop=True)
    df['trading_date'] = pd.to_datetime(df['trading_date'], format='%m/%d/%Y')
    df['contract_year'] = df['contract_year'].apply(lambda x: int('20' + x.split('^')[1] + x.split('^')[0]))
    if 'N' in df['contract_month'].values:
        month_code = 'N'
    elif 'V' in df['contract_month'].values:
        month_code = 'V'
    elif 'X' in df['contract_month'].values:
        month_code = 'X'
    elif 'Z' in df['contract_month'].values:
        month_code = 'Z'
    else:
        month_code = input("Enter the month code of the contract (N, V, X, Z): ")
    expire_dates = []
    for i in range(len(df)):
        contract_year = df['contract_year'][i]
        if month_code == 'N':
            expire_date = pd.Timestamp(str(contract_year) + '-07-01')
        elif month_code == 'V':
            expire_date = pd.Timestamp(str(contract_year) + '-10-01')
        elif month_code == 'X':
            expire_date = pd.Timestamp(str(contract_year) + '-11-01')
        elif month_code == 'Z':
            expire_date = pd.Timestamp(str(contract_year) + '-12-01')
        else:
            print("Invalid month code!")
            break
        expire_dates.append(expire_date)
        if df['trading_date'][i].year == contract_year:
            if df['trading_date'][i] > expire_date:
                df.loc[i, 'tradeability'] = 'nontradeable'
            else:
                df.loc[i, 'tradeability'] = 'tradeable'
        elif df['trading_date'][i].year != contract_year and df['trading_date'][i].month < expire_date.month:
            df.loc[i, 'tradeability'] = 'not to trade'
        else:
            df.loc[i, 'tradeability'] = 'tradeable'
    df['expire_date'] = expire_dates
    df['EMA20'] = ta.EMA(df['close'], timeperiod=20)
    df['EMA10'] = ta.EMA(df['close'], timeperiod=10)
    df['EMA5'] = ta.EMA(df['close'], timeperiod=5)
    df['ADX'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    df['BIAS'] = (df['close'] - df['close'].rolling(window=20).mean()) / df['close'].rolling(window=20).mean() * 100
    adx_values = [10, 15, 20, 25, 30, 35, 40, 45, 50]
    #bias_values = [10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]
    bias_values = [3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]
    gain_to_pain_matrix = []
    final_profits_matrix = []
    number_of_trades_matrix = []
    for adx in adx_values:
        for bias in bias_values:
            trades = []
            position = 0  
            initial_cash = 10000
            current_cash = initial_cash
            current_capital = initial_cash
            trading_cost = 0
            total_trading_cost = 0
            for i in range(1, len(df)):
                trading_date = df['trading_date'][i]
                if df['tradeability'][i] == 'tradeable':
                    if df['ADX'][i-1] > adx and \
                       df['close'][i-1] > df['EMA10'][i-1] and \
                       df['EMA10'][i-1] > df['EMA20'][i-1] and \
                       df['EMA10'][i-1] > df['EMA10'][i-2] and \
                       df['EMA20'][i-1] > df['EMA20'][i-2] and \
                       df['BIAS'][i-1] < bias and \
                       position < 3:
                        action = 'BUY'
                        price = df['open'][i]
                        position += 1
                        trading_cost = 0.0005 * price 
                        total_trading_cost += trading_cost
                        current_cash -= (price + trading_cost)
                        current_capital = current_cash + (position * df['open'][i]) 
                        trades.append([trading_date, action, price, position, trading_cost, current_capital]) 
                    elif position > 0:
                        if df['close'][i-1] < df['EMA10'][i-1] or \
                           df['BIAS'][i-1] > bias:
                           action = 'SELL'
                           price = df['open'][i]
                           current_cash += price * position
                           trading_cost = 0.0005 * price * position
                           position = 0
                           total_trading_cost += trading_cost
                           current_cash -= trading_cost
                           current_capital = current_cash
                           trades.append([trading_date, action, price, position, trading_cost, current_capital]) 
                        else:
                            action = 'HOLD'
                            price = None
                            trading_cost = 0
                            if df['close'][i] > df['close'][i-1] or \
                               df['close'][i] == df['close'][i-1]:  
                                current_cash += position * (df['close'][i] - df['close'][i-1])
                                current_capital += position * (df['close'][i] - df['close'][i-1])
                            else:
                                current_cash -= position * (df['close'][i-1] - df['close'][i])	
                                current_capital -= position * (df['close'][i-1] - df['close'][i])
                            #trades.append([trading_date, action, price, position, trading_cost, current_capital]) 
                    else:
                        action = None
                        #price = None
                        trading_cost = 0
                        #current_cash = current_capital
                else:
                    action = None
                    #price = None
                    trading_cost = 0
                    #current_cash = current_cash 
            columns = ['trading_date', 'action', 'price', 'current_share_holding', 'trading_cost', 'current_capital']
            trades_df = pd.DataFrame(trades, columns=columns)
            trades_df['daily_returns'] = trades_df['current_capital'].pct_change()
            daily_returns = trades_df['daily_returns']   
            def calculate_gain_to_pain_ratio(daily_returns):
                total_pain = sum([r if r < 0 else 0 for r in daily_returns])
                total_gain = sum([r if r > 0 else 0 for r in daily_returns])
                if total_pain == 0:
                    return float('inf')
                else:
                    return abs(total_gain / total_pain)
            gain_to_pain_ratio = calculate_gain_to_pain_ratio(daily_returns)
            gain_to_pain_matrix.append([adx, bias, gain_to_pain_ratio])
            #final_profits_matrix.append([adx, bias, (trades_df['current_capital'].iloc[len(trades_df) - 1] - initial_cash) / df['close'].mean()])
            # Check if trades_df is not empty before accessing its elements
            if not trades_df.empty:
                final_profit = (trades_df['current_capital'].iloc[-1] - initial_cash) / df['close'].mean()
            else:
                final_profit = 0.0  # or any default value you prefer
            final_profits_matrix.append([adx, bias, final_profit])
            #print("被除数：", trades_df['current_capital'].iloc[-1] - initial_cash)
            #print("除数：", df['close'].mean())
            #print("final profit:", final_profit)
            number_of_trades_matrix.append([adx, bias, len(trades_df) - 1])
    matrix_df_gain_to_pain = pd.DataFrame(gain_to_pain_matrix, columns=['ADX', 'BIAS', 'Gain-to-Pain Ratio'])
    pivot_gain_to_pain_matrix = matrix_df_gain_to_pain.pivot(index='BIAS', columns='ADX', values='Gain-to-Pain Ratio')
    matrix_df_final_profits = pd.DataFrame(final_profits_matrix, columns=['ADX', 'BIAS', 'Final Profits'])
    pivot_final_profits_matrix = matrix_df_final_profits.pivot(index='BIAS', columns='ADX', values='Final Profits')
    matrix_df_number_of_trades = pd.DataFrame(number_of_trades_matrix, columns=['ADX', 'BIAS', 'Number of Trades'])
    pivot_number_of_trades_matrix = matrix_df_number_of_trades.pivot(index='BIAS', columns='ADX', values='Number of Trades')
    # Reset index to convert pivot tables into DataFrames
    pivot_gain_to_pain_matrix_reset = pivot_gain_to_pain_matrix.reset_index()
    pivot_final_profits_matrix_reset = pivot_final_profits_matrix.reset_index()
    pivot_number_of_trades_matrix_reset = pivot_number_of_trades_matrix.reset_index()
    empty_row = pd.DataFrame([[""] * len(pivot_gain_to_pain_matrix_reset.columns)], columns=pivot_gain_to_pain_matrix_reset.columns)
    second_row = pd.DataFrame([["Final Profits"] + list(pivot_gain_to_pain_matrix_reset.columns[1:])], columns=pivot_gain_to_pain_matrix_reset.columns)
    third_row = pd.DataFrame([["Number of Trades"] + list(pivot_gain_to_pain_matrix_reset.columns[1:])], columns=pivot_gain_to_pain_matrix_reset.columns)
    combined_df = pd.concat([pivot_gain_to_pain_matrix_reset, empty_row, second_row, pivot_final_profits_matrix_reset, empty_row, third_row, pivot_number_of_trades_matrix_reset])
    combined_df.reset_index(drop=True, inplace=True)
    combined_df.columns = ['Gain to Pain'] + list(combined_df.columns[1:])
    ##########combined_df.to_csv('combined_matrix.csv', index=False, float_format='%.2f')
    # look for top 5 optimal (ADX, BIAS) pairs
    #matrix_df_gain_to_pain = pd.DataFrame(gain_to_pain_matrix, columns=['ADX', 'BIAS', 'Gain-to-Pain Ratio'])
    #matrix_df_final_profits = pd.DataFrame(final_profits_matrix, columns=['ADX', 'BIAS', 'Final Profits'])
    ranked_df = pd.merge(matrix_df_gain_to_pain, matrix_df_final_profits, on=['ADX', 'BIAS'])
    ranked_df['Gain_to_Pain_Rank'] = ranked_df['Gain-to-Pain Ratio'].rank(ascending=False)
    ranked_df['Final_Profits_Rank'] = ranked_df['Final Profits'].rank(ascending=False)
    ranked_df['Weighted_Average_Rank'] = (0.5 * ranked_df['Gain_to_Pain_Rank']) + (0.5 * ranked_df['Final_Profits_Rank'])
    sorted_df = ranked_df.sort_values(by='Weighted_Average_Rank', ascending=True)
    top_five_pairs = sorted_df.head(3)
    print(top_five_pairs[['ADX', 'BIAS', 'Gain-to-Pain Ratio', 'Final Profits']])
    # SAVE RESULTS
    optimization_folder = "optimization"
    if not os.path.exists(optimization_folder):
        os.makedirs(optimization_folder)
    combined_excel_path = os.path.join(optimization_folder, os.path.basename(file_path).replace(".csv", "_combined_matrix.xlsx"))
    combined_df.to_excel(combined_excel_path, index=False, float_format='%.4f')
    #combined_df.to_csv(os.path.join(optimization_folder, os.path.basename(file_path).replace(".csv", "_combined_matrix.csv")), index=False, float_format='%.2f')
    print(f"Processed {file_path} and saved combined matrix to {combined_excel_path}")
    # OUTPUT top 5 optimal (ADX, BIAS) pairs TO "optimization.csv"
    # Extract the optimal (ADX, BIAS) pairs from top_five_pairs DataFrame
    optimal_row = top_five_pairs.iloc[0]
    optimal_adx = optimal_row['ADX']
    optimal_bias = optimal_row['BIAS']
    gain_to_pain_optimal = optimal_row['Gain-to-Pain Ratio']
    final_profits_optimal = optimal_row['Final Profits']
    ADXBIAS_df = pd.DataFrame({
        'Code': [os.path.basename(file_path).replace(".csv", "")],
        'ADX_optimal': [optimal_adx],
        'BIAS_optimal': [optimal_bias],
        'GAIN_TO_PAIN_optimal': [gain_to_pain_optimal],
        'FINAL_PROFITS_optimal': [final_profits_optimal]
    })
    ADXBIAS_file = os.path.join(output_folder, "ADXBIAS.csv")
    if not os.path.exists(ADXBIAS_file):
        ADXBIAS_df.to_csv(ADXBIAS_file, index=False)
    else:
        ADXBIAS_df.to_csv(ADXBIAS_file, mode='a', header=False, index=False)
    print(f"Saved ADXBIAS to {ADXBIAS_file}")
if __name__ == "__main__":
    input_folder = "C:/Users/Jiaxu/Desktop/quant/data"
    output_folder = "C:/Users/Jiaxu/Desktop/quant/optimization"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_folder, filename)
            process_csv(file_path, output_folder)

In [ ]:
# ergodicity_optimize_short: FOR SHORT TREND FOLLOWING STRATEGY - OPTIMIZE (ADX, BIAS) PAIRS BASED ON GAIN-TO-PAIN RATIO, FINAL PROFITS (May-17)
# coding="utf-8"
import os
import sys
sys.path.insert(0, os.getcwd())
import talib as ta
import pandas as pd
import numpy as np
import datetime

def process_csv(file_path, output_folder):
    # LOAD DATA AND DECIDE TRADEABILITY
    df = pd.read_csv(file_path, engine='python')
    df.columns = df.columns.str.replace(' ', '')
    df.fillna(value={'open': df['close']}, inplace=True)
    df.interpolate(method='nearest', inplace=True)
    df = df.dropna(axis=1, how='all')
    df = df.iloc[::-1]
    df = df.reset_index(drop=True)
    df['trading_date'] = pd.to_datetime(df['trading_date'], format='%m/%d/%Y')
    df['contract_year'] = df['contract_year'].apply(lambda x: int('20' + x.split('^')[1] + x.split('^')[0]))
    if 'N' in df['contract_month'].values:
        month_code = 'N'
    elif 'V' in df['contract_month'].values:
        month_code = 'V'
    elif 'X' in df['contract_month'].values:
        month_code = 'X'
    elif 'Z' in df['contract_month'].values:
        month_code = 'Z'
    else:
        month_code = input("Enter the month code of the contract (N, V, X, Z): ")
    expire_dates = []
    for i in range(len(df)):
        contract_year = df['contract_year'][i]
        if month_code == 'N':
            expire_date = pd.Timestamp(str(contract_year) + '-07-01')
        elif month_code == 'V':
            expire_date = pd.Timestamp(str(contract_year) + '-10-01')
        elif month_code == 'X':
            expire_date = pd.Timestamp(str(contract_year) + '-11-01')
        elif month_code == 'Z':
            expire_date = pd.Timestamp(str(contract_year) + '-12-01')
        else:
            print("Invalid month code!")
            break
        expire_dates.append(expire_date)
        if df['trading_date'][i].year == contract_year:
            if df['trading_date'][i] > expire_date:
                df.loc[i, 'tradeability'] = 'nontradeable'
            else:
                df.loc[i, 'tradeability'] = 'tradeable'
        elif df['trading_date'][i].year != contract_year and df['trading_date'][i].month < expire_date.month:
            df.loc[i, 'tradeability'] = 'not to trade'
        else:
            df.loc[i, 'tradeability'] = 'tradeable'
    # LONG TREND FOLLWOING STRATEGY
    df['expire_date'] = expire_dates
    df['EMA20'] = ta.EMA(df['close'], timeperiod=20)
    df['EMA10'] = ta.EMA(df['close'], timeperiod=10)
    df['EMA5'] = ta.EMA(df['close'], timeperiod=5)
    df['ADX'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    df['BIAS'] = (df['close'] - df['close'].rolling(window=20).mean()) / df['close'].rolling(window=20).mean() * 100
    adx_values = [10, 15, 20, 25, 30, 35, 40, 45]
    #bias_values = [10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]
    bias_values = [-3, -4, -5, -6, -7, -8, -9, -10, -12, -14, -16, -18, -20, -22, -24, -26, -28, -30]
    gain_to_pain_matrix = []
    final_profits_matrix = []
    number_of_trades_matrix = []
    for adx in adx_values:
        for bias in bias_values:
            trades = []
            position = 0  
            initial_cash = 10000
            current_cash = initial_cash
            current_capital = initial_cash
            trading_cost = 0
            total_trading_cost = 0
            for i in range(1, len(df)):
                trading_date = df['trading_date'][i]
                if df['tradeability'][i] == 'tradeable':
                    if df['ADX'][i-1] > adx and \
                       df['close'][i-1] < df['EMA10'][i-1] and \
                       df['EMA10'][i-1] < df['EMA20'][i-1] and \
                       df['EMA10'][i-1] < df['EMA10'][i-2] and \
                       df['EMA20'][i-1] < df['EMA20'][i-2] and \
                       df['BIAS'][i-1] > bias and \
                       position < 1:
                        action = 'SELL'
                        price = df['open'][i]
                        position += 1
                        trading_cost = 0.0005 * price 
                        total_trading_cost += trading_cost
                        current_cash -= (price + trading_cost)
                        current_capital = current_cash + (position * df['open'][i]) 
                        trades.append([trading_date, action, price, position, trading_cost, current_capital]) 
                    elif position > 0:
                        if df['close'][i-1] > df['EMA10'][i-1] or \
                           df['BIAS'][i-1] < bias:
                           action = 'BUY'
                           price = df['open'][i]
                           current_cash += price * position
                           trading_cost = 0.0005 * price * position
                           position = 0
                           total_trading_cost += trading_cost
                           current_cash -= trading_cost
                           current_capital = current_cash
                           trades.append([trading_date, action, price, position, trading_cost, current_capital]) 
                        else:
                            action = 'HOLD'
                            price = None
                            trading_cost = 0
                            if df['close'][i] < df['close'][i-1] or \
                               df['close'][i] == df['close'][i-1]:  
                                current_cash += position * (df['close'][i-1] - df['close'][i])
                                current_capital += position * (df['close'][i-1] - df['close'][i])
                            else:
                                current_cash -= position * (df['close'][i] - df['close'][i-1])	
                                current_capital -= position * (df['close'][i] - df['close'][i-1])
                            #trades.append([trading_date, action, price, position, trading_cost, current_capital]) 
                    else:
                        action = None
                        #price = None
                        trading_cost = 0
                        #current_cash = current_capital
                else:
                    action = None
                    #price = None
                    trading_cost = 0
                    #current_cash = current_cash 
            columns = ['trading_date', 'action', 'price', 'current_share_holding', 'trading_cost', 'current_capital']
            trades_df = pd.DataFrame(trades, columns=columns)
            trades_df['daily_returns'] = trades_df['current_capital'].pct_change()
            daily_returns = trades_df['daily_returns']   
            def calculate_gain_to_pain_ratio(daily_returns):
                total_pain = sum([r if r < 0 else 0 for r in daily_returns])
                total_gain = sum([r if r > 0 else 0 for r in daily_returns])
                if total_pain == 0:
                    return float('inf')
                else:
                    return abs(total_gain / total_pain)
            gain_to_pain_ratio = calculate_gain_to_pain_ratio(daily_returns)
            gain_to_pain_matrix.append([adx, bias, gain_to_pain_ratio])
            #final_profits_matrix.append([adx, bias, (trades_df['current_capital'].iloc[len(trades_df) - 1] - initial_cash) / df['close'].mean()])
            # Check if trades_df is not empty before accessing its elements
            if not trades_df.empty:
                final_profit = (trades_df['current_capital'].iloc[-1] - initial_cash) / df['close'].mean()
            else:
                final_profit = 0.0  # or any default value you prefer
            final_profits_matrix.append([adx, bias, final_profit])
            #print("被除数：", trades_df['current_capital'].iloc[-1] - initial_cash)
            #print("除数：", df['close'].mean())
            #print("final profit:", final_profit)
            number_of_trades_matrix.append([adx, bias, len(trades_df) - 1])
    matrix_df_gain_to_pain = pd.DataFrame(gain_to_pain_matrix, columns=['ADX', 'BIAS', 'Gain-to-Pain Ratio'])
    pivot_gain_to_pain_matrix = matrix_df_gain_to_pain.pivot(index='BIAS', columns='ADX', values='Gain-to-Pain Ratio')
    matrix_df_final_profits = pd.DataFrame(final_profits_matrix, columns=['ADX', 'BIAS', 'Final Profits'])
    pivot_final_profits_matrix = matrix_df_final_profits.pivot(index='BIAS', columns='ADX', values='Final Profits')
    matrix_df_number_of_trades = pd.DataFrame(number_of_trades_matrix, columns=['ADX', 'BIAS', 'Number of Trades'])
    pivot_number_of_trades_matrix = matrix_df_number_of_trades.pivot(index='BIAS', columns='ADX', values='Number of Trades')
    # Reset index to convert pivot tables into DataFrames
    pivot_gain_to_pain_matrix_reset = pivot_gain_to_pain_matrix.reset_index()
    pivot_final_profits_matrix_reset = pivot_final_profits_matrix.reset_index()
    pivot_number_of_trades_matrix_reset = pivot_number_of_trades_matrix.reset_index()
    empty_row = pd.DataFrame([[""] * len(pivot_gain_to_pain_matrix_reset.columns)], columns=pivot_gain_to_pain_matrix_reset.columns)
    second_row = pd.DataFrame([["Final Profits"] + list(pivot_gain_to_pain_matrix_reset.columns[1:])], columns=pivot_gain_to_pain_matrix_reset.columns)
    third_row = pd.DataFrame([["Number of Trades"] + list(pivot_gain_to_pain_matrix_reset.columns[1:])], columns=pivot_gain_to_pain_matrix_reset.columns)
    combined_df = pd.concat([pivot_gain_to_pain_matrix_reset, empty_row, second_row, pivot_final_profits_matrix_reset, empty_row, third_row, pivot_number_of_trades_matrix_reset])
    combined_df.reset_index(drop=True, inplace=True)
    combined_df.columns = ['Gain to Pain'] + list(combined_df.columns[1:])
    ##########combined_df.to_csv('combined_matrix.csv', index=False, float_format='%.2f')
    # look for top 5 optimal (ADX, BIAS) pairs
    #matrix_df_gain_to_pain = pd.DataFrame(gain_to_pain_matrix, columns=['ADX', 'BIAS', 'Gain-to-Pain Ratio'])
    #matrix_df_final_profits = pd.DataFrame(final_profits_matrix, columns=['ADX', 'BIAS', 'Final Profits'])
    ranked_df = pd.merge(matrix_df_gain_to_pain, matrix_df_final_profits, on=['ADX', 'BIAS'])
    ranked_df['Gain_to_Pain_Rank'] = ranked_df['Gain-to-Pain Ratio'].rank(ascending=False)
    ranked_df['Final_Profits_Rank'] = ranked_df['Final Profits'].rank(ascending=False)
    ranked_df['Weighted_Average_Rank'] = (0.5 * ranked_df['Gain_to_Pain_Rank']) + (0.5 * ranked_df['Final_Profits_Rank'])
    sorted_df = ranked_df.sort_values(by='Weighted_Average_Rank', ascending=True)
    top_five_pairs = sorted_df.head(3)
    print(top_five_pairs[['ADX', 'BIAS', 'Gain-to-Pain Ratio', 'Final Profits']])
    # SAVE RESULTS
    optimization_folder = "optimization"
    if not os.path.exists(optimization_folder):
        os.makedirs(optimization_folder)
    combined_excel_path = os.path.join(optimization_folder, os.path.basename(file_path).replace(".csv", "_combined_matrix.xlsx"))
    combined_df.to_excel(combined_excel_path, index=False, float_format='%.4f')
    #combined_df.to_csv(os.path.join(optimization_folder, os.path.basename(file_path).replace(".csv", "_combined_matrix.csv")), index=False, float_format='%.2f')
    print(f"Processed {file_path} and saved combined matrix to {combined_excel_path}")
    # OUTPUT top 5 optimal (ADX, BIAS) pairs TO "optimization.csv"
    # Extract the optimal (ADX, BIAS) pairs from top_five_pairs DataFrame
    optimal_row = top_five_pairs.iloc[0]
    optimal_adx = optimal_row['ADX']
    optimal_bias = optimal_row['BIAS']
    gain_to_pain_optimal = optimal_row['Gain-to-Pain Ratio']
    final_profits_optimal = optimal_row['Final Profits']
    ADXBIAS_df = pd.DataFrame({
        'Code': [os.path.basename(file_path).replace(".csv", "")],
        'ADX_optimal': [optimal_adx],
        'BIAS_optimal': [optimal_bias],
        'GAIN_TO_PAIN_optimal': [gain_to_pain_optimal],
        'FINAL_PROFITS_optimal': [final_profits_optimal]
    })
    ADXBIAS_file = os.path.join(output_folder, "ADXBIAS.csv")
    if not os.path.exists(ADXBIAS_file):
        ADXBIAS_df.to_csv(ADXBIAS_file, index=False)
    else:
        ADXBIAS_df.to_csv(ADXBIAS_file, mode='a', header=False, index=False)
    print(f"Saved ADXBIAS to {ADXBIAS_file}")
if __name__ == "__main__":
    input_folder = "C:/Users/Jiaxu/Desktop/quant/data"
    output_folder = "C:/Users/Jiaxu/Desktop/quant/optimization"
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_folder, filename)
            process_csv(file_path, output_folder)

In [ ]:
# CTA_ver0_long: IMPLEMENT LONG TREND FOLLOWING STRATEGY WITH OPTIMAL (ADX, BIAS) PAIR FOR A SINGLE CONTRACT TO GENERATE TRADE RECORDS, DAILY PNLS, AND PERFORMANCE METRICS (May-13)
# coding="utf-8"
import os
import sys
sys.path.insert(0, os.getcwd())
import talib as ta
import pandas as pd
import numpy as np
import datetime

def process_csv(file_path, result_folder):
    # LOAD DATA AND DECIDE TRADEABILITY
    df = pd.read_csv(file_path, engine='python')
    df.columns = df.columns.str.replace(' ', '')
    df.fillna(value={'open': df['close']}, inplace=True)
    df.interpolate(method='nearest', inplace=True)
    df = df.dropna(axis=1, how='all')
    df = df.iloc[::-1]
    df = df.reset_index(drop=True)
    df['trading_date'] = pd.to_datetime(df['trading_date'], format='%m/%d/%Y')
    df['contract_year'] = df['contract_year'].apply(lambda x: int('20' + x.split('^')[1] + x.split('^')[0]))
    if 'N' in df['contract_month'].values:
        month_code = 'N'
    elif 'V' in df['contract_month'].values:
        month_code = 'V'
    elif 'X' in df['contract_month'].values:
        month_code = 'X'
    elif 'Z' in df['contract_month'].values:
        month_code = 'Z'
    else:
        month_code = input("Enter the month code of the contract (N, V, X, Z): ")
    expire_dates = []
    for i in range(len(df)):
        contract_year = df['contract_year'][i]
        if month_code == 'N':
            expire_date = pd.Timestamp(str(contract_year) + '-07-01')
        elif month_code == 'V':
            expire_date = pd.Timestamp(str(contract_year) + '-10-01')
        elif month_code == 'X':
            expire_date = pd.Timestamp(str(contract_year) + '-11-01')
        elif month_code == 'Z':
            expire_date = pd.Timestamp(str(contract_year) + '-12-01')
        else:
            print("Invalid month code!")
            break
        expire_dates.append(expire_date)

        if df['trading_date'][i].year == contract_year:
            if df['trading_date'][i] > expire_date or df['trading_date'][i] == expire_date:
                df.loc[i, 'tradeability'] = 'nontradeable'
            else:
                df.loc[i, 'tradeability'] = 'tradeable'
        elif df['trading_date'][i].year != contract_year and df['trading_date'][i].month < expire_date.month:
            df.loc[i, 'tradeability'] = 'not to trade'
        else:
            df.loc[i, 'tradeability'] = 'tradeable'
    # Save tradeability information
    tradeability_path = os.path.join(result_folder, "tradeability.csv")
    df['expire_date'] = expire_dates
    df.to_csv(tradeability_path, index=False)
    print(f"Tradeability information saved to {tradeability_path}")

    # LONG TREND FOLLWOING STRATEGY
    # LONG CTZ WITH (ADX, BIAS) = (35, 10) AS AN EXAMPLE
    df['expire_date'] = expire_dates
    df['EMA20'] = ta.EMA(df['close'], timeperiod=20)
    df['EMA10'] = ta.EMA(df['close'], timeperiod=10)
    df['EMA5'] = ta.EMA(df['close'], timeperiod=5)
    df['ADX'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    df['BIAS'] = (df['close'] - df['close'].rolling(window=20).mean()) / df['close'].rolling(window=20).mean() * 100
    position = 0  
    initial_cash = 10000
    current_cash = initial_cash
    current_capital = initial_cash 
    trading_cost = 0  
    total_trading_cost = 0
    action = None
    trades = []
    daily = []
    for i in range(1, len(df)):
        trading_date = df['trading_date'][i]
        if df['tradeability'][i] == 'tradeable':
            if df['ADX'][i-1] > 15 and \
               df['close'][i-1] > df['EMA10'][i-1] and \
               df['EMA10'][i-1] > df['EMA20'][i-1] and \
               df['EMA10'][i-1] > df['EMA10'][i-2] and \
               df['EMA20'][i-1] > df['EMA20'][i-2] and \
               df['BIAS'][i-1] < 16 and \
               position < 3:
                action = 'BUY'
                price = df['open'][i]
                position += 1
                trading_cost = 0.0005 * price 
                total_trading_cost += trading_cost
                current_cash -= (price + trading_cost)
                current_capital = current_cash + (position * df['open'][i])
                trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
            elif position > 0:
                if df['close'][i-1] < df['EMA10'][i-1] or \
                   df['BIAS'][i-1] > 16:
                    action = 'SELL'
                    price = df['open'][i]
                    current_cash += price * position
                    trading_cost = 0.0005 * price * position
                    position = 0
                    total_trading_cost += trading_cost
                    current_cash -= trading_cost
                    current_capital = current_cash
                    trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                else:
                    action = 'HOLD'
                    price = None
                    trading_cost = 0
                    if df['close'][i] > df['close'][i-1] or \
                       df['close'][i] == df['close'][i-1]:  
                        current_cash += position * (df['close'][i] - df['close'][i-1])
                        current_capital += position * (df['close'][i] - df['close'][i-1])
                    else:
                        current_cash -= position * (df['close'][i-1] - df['close'][i])	
                        current_capital -= position * (df['close'][i-1] - df['close'][i])
                    #trades.append([trading_date, action, price, position, trading_cost, current_capital]) 
            else:
                action = None
                #price = None
                trading_cost = 0
                #current_cash = current_capital
        #else:
            #action = None
            #price = None
            #trading_cost = 0
            #current_cash = current_cash
            daily_capital = current_cash + (position * df['close'][i])
        #daily_capital = current_capital + position * (df['close'][i] - df['open'][i])
            daily.append([trading_date, df['close'][i], action, df['open'][i], position, trading_cost, current_cash, current_capital, daily_capital]) 
            trades_df = pd.DataFrame(trades, columns=['trading_date', 'action', 'price', 'current_share_holding', 'trading_cost', 'current_cash', 'current_capital'])
            daily_df = pd.DataFrame(daily, columns=['date', 'close', 'action', 'open', 'current_share_holding', 'trading_cost', 'current_cash', 'current_capital', 'daily_capital'])
    # CALCULATE PERFORMANCE METRICS PER TRADE AND DAILY
    trades_df['per_trade_returns'] = trades_df['current_capital'].pct_change()
    daily_df['daily_returns'] = daily_df['daily_capital'].pct_change()
    sharpe_ratio_per_trade = trades_df['per_trade_returns'].mean() / trades_df['per_trade_returns'].std()
    sharpe_ratio_daily = daily_df['daily_returns'].mean() / daily_df['daily_returns'].std()
    max_drawdown_per_trade = (1 - (trades_df['current_capital'] / trades_df['current_capital'].cummax())).max()
    max_drawdown_daily = (1 - (daily_df['daily_capital'] / daily_df['daily_capital'].cummax())).max()
    per_trade_returns = trades_df['per_trade_returns']
    def calculate_gain_to_pain_ratio_per_trade(per_trade_returns):
        total_pain_per_trade = sum([r if r < 0 else 0 for r in per_trade_returns])
        total_gain_per_trade = sum([r if r > 0 else 0 for r in per_trade_returns])
        if total_pain_per_trade == 0:
            return float('inf')  
        else:
            return abs(total_gain_per_trade / total_pain_per_trade)
    gain_to_pain_ratio_per_trade = calculate_gain_to_pain_ratio_per_trade(per_trade_returns)
    daily_returns = daily_df['daily_returns']
    def calculate_gain_to_pain_ratio_daily(daily_returns): 
        total_pain_daily = sum([r if r < 0 else 0 for r in daily_returns])
        total_gain_daily = sum([r if r > 0 else 0 for r in daily_returns])
        if total_pain_daily == 0:
            return float('inf') 
        else:
            return abs(total_gain_daily / total_pain_daily)
    gain_to_pain_ratio_daily = calculate_gain_to_pain_ratio_daily(daily_returns)
    #final_profit_per_trade = (trades_df['current_capital'].iloc[-1] - initial_cash) / df['close'].mean()
    final_profit = (daily_df['daily_capital'].iloc[-1] - initial_cash) / df['close'].mean()
    trades_df['sharpe_ratio_per_trade'] = sharpe_ratio_per_trade
    trades_df['max_drawdown_per_trade'] = max_drawdown_per_trade
    trades_df['gain_to_pain_ratio_per_trade'] = gain_to_pain_ratio_per_trade
    #trades_df['final_profit_per_trade'] = final_profit_per_trade
    daily_df['sharpe_ratio_daily'] = sharpe_ratio_daily
    daily_df['max_drawdown_daily'] = max_drawdown_daily
    daily_df['gain_to_pain_ratio_daily'] = gain_to_pain_ratio_daily
    daily_df['final_profit_daily'] = final_profit
    # SAVE RESULTS
    result_folder = "result"
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)
    #tradeability_path = os.path.join(result_folder, "tradeability.csv")
    #df.to_csv(tradeability_path, index=False)
    trades_excel_path = os.path.join(result_folder, os.path.basename(file_path).replace(".csv", "_trades.xlsx"))
    trades_df.to_excel(trades_excel_path, index=False, float_format='%.4f')
    #trades_df.to_csv(os.path.join(result_folder, os.path.basename(file_path).replace(".csv", "_trades.csv")), index=False)
    daily_excel_path = os.path.join(result_folder, os.path.basename(file_path).replace(".csv", "_daily.xlsx"))
    daily_df.to_excel(daily_excel_path, index=False, float_format='%.4f')
    print(f"Processed {file_path} and saved trade record to {result_folder}")
    print("Sharpe Ratio Per Trade:", sharpe_ratio_per_trade)
    print("Max Drawdown Per Trade:", max_drawdown_per_trade)
    print("Gain-to-Pain Ratio Per Trade:", gain_to_pain_ratio_per_trade)
    #print("Final Profit Per Trade:", final_profit_per_trade)
    print("Total Trading Cost:", total_trading_cost)
    print(f"Processed {file_path} and saved daily record to {result_folder}")
    print("Sharpe Ratio Daily:", sharpe_ratio_daily)
    print("Max Drawdown Daily:", max_drawdown_daily)
    print("Gain-to-Pain Ratio Daily:", gain_to_pain_ratio_daily)
    print("Final Profit:", final_profit)
    # OUTPUT PERFORMANCE METRICS PER TRADE TO "performance_metrics_per_trade.csv"
    performance_metrics_per_trade_df = pd.DataFrame({
        'Code': [os.path.basename(file_path).replace(".csv", "")],
        'Sharpe_Ratio_Per_Trade': [sharpe_ratio_per_trade],
        'Max_Drawdown_Per_Trade': [max_drawdown_per_trade],
        'Gain_to_Pain_Ratio_Per_Trade': [gain_to_pain_ratio_per_trade],
        #'Final_Profit_Per_Trade': [final_profit_per_trade]
    })
    performance_metrics_per_trade_file = os.path.join(result_folder, "performance_metrics_per_trade.csv")
    if not os.path.exists(performance_metrics_per_trade_file):
        performance_metrics_per_trade_df.to_csv(performance_metrics_per_trade_file, index=False)
    else:
        performance_metrics_per_trade_df.to_csv(performance_metrics_per_trade_file, mode='a', header=False, index=False)
    print(f"Saved performance metrics per trade to {performance_metrics_per_trade_file}")
    # OUTPUT PERFORMANCE METRICS DAILY TO "performance_metrics_daily.csv"
    performance_metrics_daily_df = pd.DataFrame({
        'Code': [os.path.basename(file_path).replace(".csv", "")],
        'Sharpe_Ratio_Daily': [sharpe_ratio_daily],
        'Max_Drawdown_Daily': [max_drawdown_daily],
        'Gain_to_Pain_Ratio_Daily': [gain_to_pain_ratio_daily],
        'Final_Profit': [final_profit]
    })
    performance_metrics_daily_file = os.path.join(result_folder, "performance_metrics_daily.csv")
    if not os.path.exists(performance_metrics_daily_file):
        performance_metrics_daily_df.to_csv(performance_metrics_daily_file, index=False)
    else:
        performance_metrics_daily_df.to_csv(performance_metrics_daily_file, mode='a', header=False, index=False)
    print(f"Saved performance metrics daily to {performance_metrics_daily_file}")
if __name__ == "__main__":
    input_folder = "C:/Users/Jiaxu/Desktop/quant/test"
    result_folder = "C:/Users/Jiaxu/Desktop/quant/result"
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_folder, filename)
            process_csv(file_path, result_folder)

In [ ]:
# CTA_ver0_short: IMPLEMENT SHORT TREND FOLLOWING STRATEGY WITH OPTIMAL (ADX, BIAS) PAIR FOR A SINGLE CONTRACT TO GENERATE TRADE RECORDS, DAILY PNLS, AND PERFORMANCE METRICS (May-14)
# coding="utf-8"
import os
import sys
sys.path.insert(0, os.getcwd())
import talib as ta
import pandas as pd
import numpy as np
import datetime

def process_csv(file_path):
    # LOAD DATA AND DECIDE TRADEABILITY
    df = pd.read_csv(file_path, engine='python')
    df.columns = df.columns.str.replace(' ', '')
    df.fillna(value={'open': df['close']}, inplace=True)
    df.interpolate(method='nearest', inplace=True)
    df = df.dropna(axis=1, how='all')
    df = df.iloc[::-1]
    df = df.reset_index(drop=True)
    df['trading_date'] = pd.to_datetime(df['trading_date'], format='%m/%d/%Y')
    df['contract_year'] = df['contract_year'].apply(lambda x: int('20' + x.split('^')[1] + x.split('^')[0]))
    if 'N' in df['contract_month'].values:
        month_code = 'N'
    elif 'V' in df['contract_month'].values:
        month_code = 'V'
    elif 'X' in df['contract_month'].values:
        month_code = 'X'
    elif 'Z' in df['contract_month'].values:
        month_code = 'Z'
    else:
        month_code = input("Enter the month code of the contract (N, V, X, Z): ")
    expire_dates = []
    for i in range(len(df)):
        contract_year = df['contract_year'][i]
        if month_code == 'N':
            expire_date = pd.Timestamp(str(contract_year) + '-07-01')
        elif month_code == 'V':
            expire_date = pd.Timestamp(str(contract_year) + '-10-01')
        elif month_code == 'X':
            expire_date = pd.Timestamp(str(contract_year) + '-11-01')
        elif month_code == 'Z':
            expire_date = pd.Timestamp(str(contract_year) + '-12-01')
        else:
            print("Invalid month code!")
            break
        expire_dates.append(expire_date)
        if df['trading_date'][i].year == contract_year:
            if df['trading_date'][i] > expire_date:
                df.loc[i, 'tradeability'] = 'nontradeable'
            else:
                df.loc[i, 'tradeability'] = 'tradeable'
        elif df['trading_date'][i].year != contract_year and df['trading_date'][i].month < expire_date.month:
            df.loc[i, 'tradeability'] = 'not to trade'
        else:
            df.loc[i, 'tradeability'] = 'tradeable'
    df['expire_date'] = expire_dates
    df['EMA20'] = ta.EMA(df['close'], timeperiod=20)
    df['EMA10'] = ta.EMA(df['close'], timeperiod=10)
    df['EMA5'] = ta.EMA(df['close'], timeperiod=5)
    df['ADX'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    df['BIAS'] = (df['close'] - df['close'].rolling(window=20).mean()) / df['close'].rolling(window=20).mean() * 100
    position = 0  
    initial_cash = 10000
    current_cash = initial_cash
    current_capital = initial_cash 
    trading_cost = 0  
    total_trading_cost = 0
    action = None
    trades = []
    daily = []
    # SHORT TREND FOLLWOING STRATEGY
    # SHORT KCN WITH (ADX, BIAS) = (30, -8) AS AN EXAMPLE
    for i in range(1, len(df)):
        trading_date = df['trading_date'][i]
        if df['tradeability'][i] == 'tradeable':
            if df['ADX'][i-1] > 30 and \
               df['close'][i-1] < df['EMA10'][i-1] and \
               df['EMA10'][i-1] < df['EMA20'][i-1] and \
               df['EMA10'][i-1] < df['EMA10'][i-2] and \
               df['EMA20'][i-1] < df['EMA20'][i-2] and \
               df['BIAS'][i-1] > -12 and \
               position < 1:
                action = 'SELL'
                price = df['open'][i]
                position += 1
                trading_cost = 0.0005 * price 
                total_trading_cost += trading_cost
                current_cash -= (price + trading_cost)
                current_capital = current_cash + (position * df['open'][i])
                trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
            elif position > 0:
                if df['close'][i-1] > df['EMA10'][i-1] or \
                   df['BIAS'][i-1] < -12:
                    action = 'BUY'
                    price = df['open'][i]
                    current_cash += price * position
                    trading_cost = 0.0005 * price * position
                    position = 0
                    total_trading_cost += trading_cost
                    current_cash -= trading_cost
                    current_capital = current_cash
                    trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                else:
                    action = 'HOLD'
                    price = None
                    trading_cost = 0
                    if df['close'][i] < df['close'][i-1] or \
                       df['close'][i] == df['close'][i-1]:  
                        current_cash += position * (df['close'][i-1] - df['close'][i])
                        current_capital += position * (df['close'][i-1] - df['close'][i])
                    else:
                        current_cash -= position * (df['close'][i] - df['close'][i-1])	
                        current_capital -= position * (df['close'][i] - df['close'][i-1])
                    #trades.append([trading_date, action, price, position, trading_cost, current_capital]) 
            else:
                action = None
                #price = None
                trading_cost = 0
                #current_cash = current_capital
        #else:
            #action = None
            #price = None
            #trading_cost = 0
            #current_cash = current_cash
            daily_capital = current_cash + (position * df['close'][i])
            #daily_capital = current_capital + position * (df['close'][i] - df['open'][i])
            daily.append([trading_date, df['close'][i], action, df['open'][i], position, trading_cost, current_cash, current_capital, daily_capital]) 
    trades_df = pd.DataFrame(trades, columns=['trading_date', 'action', 'price', 'current_share_holding', 'trading_cost', 'current_cash', 'current_capital'])
    daily_df = pd.DataFrame(daily, columns=['date', 'close', 'action', 'open', 'current_share_holding', 'trading_cost', 'current_cash', 'current_capital', 'daily_capital'])
    # CALCULATE PERFORMANCE METRICS PER TRADE AND DAILY
    trades_df['per_trade_returns'] = trades_df['current_capital'].pct_change()
    daily_df['daily_returns'] = daily_df['daily_capital'].pct_change()
    sharpe_ratio_per_trade = trades_df['per_trade_returns'].mean() / trades_df['per_trade_returns'].std()
    sharpe_ratio_daily = daily_df['daily_returns'].mean() / daily_df['daily_returns'].std()
    max_drawdown_per_trade = (1 - (trades_df['current_capital'] / trades_df['current_capital'].cummax())).max()
    max_drawdown_daily = (1 - (daily_df['daily_capital'] / daily_df['daily_capital'].cummax())).max()
    per_trade_returns = trades_df['per_trade_returns']
    def calculate_gain_to_pain_ratio_per_trade(per_trade_returns):
        total_pain_per_trade = sum([r if r < 0 else 0 for r in per_trade_returns])
        total_gain_per_trade = sum([r if r > 0 else 0 for r in per_trade_returns])
        if total_pain_per_trade == 0:
            return float('inf')  
        else:
            return abs(total_gain_per_trade / total_pain_per_trade)
    gain_to_pain_ratio_per_trade = calculate_gain_to_pain_ratio_per_trade(per_trade_returns)
    daily_returns = daily_df['daily_returns']
    def calculate_gain_to_pain_ratio_daily(daily_returns): 
        total_pain_daily = sum([r if r < 0 else 0 for r in daily_returns])
        total_gain_daily = sum([r if r > 0 else 0 for r in daily_returns])
        if total_pain_daily == 0:
            return float('inf') 
        else:
            return abs(total_gain_daily / total_pain_daily)
    gain_to_pain_ratio_daily = calculate_gain_to_pain_ratio_daily(daily_returns)
    #final_profit_per_trade = (trades_df['current_capital'].iloc[-1] - initial_cash) / df['close'].mean()
    final_profit = (daily_df['daily_capital'].iloc[-1] - initial_cash) / df['close'].mean()
    trades_df['sharpe_ratio_per_trade'] = sharpe_ratio_per_trade
    trades_df['max_drawdown_per_trade'] = max_drawdown_per_trade
    trades_df['gain_to_pain_ratio_per_trade'] = gain_to_pain_ratio_per_trade
    #trades_df['final_profit_per_trade'] = final_profit_per_trade
    daily_df['sharpe_ratio_daily'] = sharpe_ratio_daily
    daily_df['max_drawdown_daily'] = max_drawdown_daily
    daily_df['gain_to_pain_ratio_daily'] = gain_to_pain_ratio_daily
    daily_df['final_profit'] = final_profit
    # SAVE RESULTS
    result_folder = "result"
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)
    tradeability_path = os.path.join(result_folder, "tradeability.csv")
    df.to_csv(tradeability_path, index=False)
    trades_excel_path = os.path.join(result_folder, os.path.basename(file_path).replace(".csv", "_trades.xlsx"))
    trades_df.to_excel(trades_excel_path, index=False, float_format='%.4f')
    #trades_df.to_csv(os.path.join(result_folder, os.path.basename(file_path).replace(".csv", "_trades.csv")), index=False)
    daily_excel_path = os.path.join(result_folder, os.path.basename(file_path).replace(".csv", "_daily.xlsx"))
    daily_df.to_excel(daily_excel_path, index=False, float_format='%.4f')
    print(f"Processed {file_path} and saved trade record to {result_folder}")
    print("Sharpe Ratio Per Trade:", sharpe_ratio_per_trade)
    print("Max Drawdown Per Trade:", max_drawdown_per_trade)
    print("Gain-to-Pain Ratio Per Trade:", gain_to_pain_ratio_per_trade)
    #print("Final Profit Per Trade:", final_profit_per_trade)
    print("Total Trading Cost:", total_trading_cost)
    print(f"Processed {file_path} and saved daily record to {result_folder}")
    print("Sharpe Ratio Daily:", sharpe_ratio_daily)
    print("Max Drawdown Daily:", max_drawdown_daily)
    print("Gain-to-Pain Ratio Daily:", gain_to_pain_ratio_daily)
    print("Final Profit:", final_profit)
    # OUTPUT PERFORMANCE METRICS PER TRADE TO "performance_metrics_per_trade.csv"
    performance_metrics_per_trade_df = pd.DataFrame({
        'Code': [os.path.basename(file_path).replace(".csv", "")],
        'Sharpe_Ratio_Per_Trade': [sharpe_ratio_per_trade],
        'Max_Drawdown_Per_Trade': [max_drawdown_per_trade],
        'Gain_to_Pain_Ratio_Per_Trade': [gain_to_pain_ratio_per_trade],
        #'Final_Profit_Per_Trade': [final_profit_per_trade]
    })
    performance_metrics_per_trade_file = os.path.join(result_folder, "performance_metrics_per_trade.csv")
    if not os.path.exists(performance_metrics_per_trade_file):
        performance_metrics_per_trade_df.to_csv(performance_metrics_per_trade_file, index=False)
    else:
        performance_metrics_per_trade_df.to_csv(performance_metrics_per_trade_file, mode='a', header=False, index=False)
    print(f"Saved performance metrics per trade to {performance_metrics_per_trade_file}")
    # OUTPUT PERFORMANCE METRICS DAILY TO "performance_metrics_daily.csv"
    performance_metrics_daily_df = pd.DataFrame({
        'Code': [os.path.basename(file_path).replace(".csv", "")],
        'Sharpe_Ratio_Daily': [sharpe_ratio_daily],
        'Max_Drawdown_Daily': [max_drawdown_daily],
        'Gain_to_Pain_Ratio_Daily': [gain_to_pain_ratio_daily],
        'Final_Profit': [final_profit]
    })
    performance_metrics_daily_file = os.path.join(result_folder, "performance_metrics_daily.csv")
    if not os.path.exists(performance_metrics_daily_file):
        performance_metrics_daily_df.to_csv(performance_metrics_daily_file, index=False)
    else:
        performance_metrics_daily_df.to_csv(performance_metrics_daily_file, mode='a', header=False, index=False)
    print(f"Saved performance metrics daily to {performance_metrics_daily_file}")
if __name__ == "__main__":
    input_folder = "C:/Users/Jiaxu/Desktop/quant/test"
    result_folder = "C:/Users/Jiaxu/Desktop/quant/result"
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_folder, filename)
            process_csv(file_path)

In [ ]:
# CTA_ver1: IMPLEMENT UNI-DIRACTIONAL CTA STRATEGY WITH OPTIMAL (ADX, BIAS) PAIRS FOR LISTED CONTRACTS TO GENERATE TRADE RECORDS, DAILY PNLS, AND PERFORMANCE METRICS
# coding="utf-8"
import os
import sys
sys.path.insert(0, os.getcwd())
import talib as ta
import pandas as pd
import numpy as np
import datetime
# READ STRATEGY FILE
def read_strategy(strategy_file):
    strategy_df = pd.read_csv(strategy_file)
    return strategy_df
# PROCESS HISTROICAL DATA FILES
def process_csv(file_path, strategy_df):
    # EXTRACT CTA STRATEGY INFORMATION FROM quant/CTA_data/instruction/CTA_strategy.csv
    contract_code = os.path.basename(file_path).replace(".csv", "")
    print("Contract Code:", contract_code)
    contract_strategy = strategy_df[strategy_df['CODE'] == contract_code].iloc[0]
    print(contract_strategy)
    if contract_strategy['RUN_OR_NOT'] != 'RUN':
        print(f"Skipping {file_path} as per strategy instruction.")
        return
    adx_optimal = contract_strategy['ADX_OPTIMAL']
    bias_optimal = contract_strategy['BIAS_OPTIMAL']
    # LOAD DATA AND DECIDE TRADEABILITY
    df = pd.read_csv(file_path, engine='python')
    df.columns = df.columns.str.replace(' ', '')
    df.fillna(value={'open': df['close']}, inplace=True)
    df.interpolate(method='nearest', inplace=True)
    df = df.dropna(axis=1, how='all')
    df = df.iloc[::-1]
    df = df.reset_index(drop=True)
    df['trading_date'] = pd.to_datetime(df['trading_date'], format='%m/%d/%Y')
    df['contract_year'] = df['contract_year'].apply(lambda x: int('20' + x.split('^')[1] + x.split('^')[0]))
    if 'N' in df['contract_month'].values:
        month_code = 'N'
    elif 'V' in df['contract_month'].values:
        month_code = 'V'
    elif 'X' in df['contract_month'].values:
        month_code = 'X'
    elif 'Z' in df['contract_month'].values:
        month_code = 'Z'
    else:
        month_code = input("Enter the month code of the contract (N, V, X, Z): ")
    expire_dates = []
    for i in range(len(df)):
        contract_year = df['contract_year'][i]
        if month_code == 'N':
            expire_date = pd.Timestamp(str(contract_year) + '-07-01')
        elif month_code == 'V':
            expire_date = pd.Timestamp(str(contract_year) + '-10-01')
        elif month_code == 'X':
            expire_date = pd.Timestamp(str(contract_year) + '-11-01')
        elif month_code == 'Z':
            expire_date = pd.Timestamp(str(contract_year) + '-12-01')
        else:
            print("Invalid month code!")
            break
        expire_dates.append(expire_date)
        if df['trading_date'][i].year == contract_year:
            if df['trading_date'][i] > expire_date:
                df.loc[i, 'tradeability'] = 'nontradeable'
            else:
                df.loc[i, 'tradeability'] = 'tradeable'
        elif df['trading_date'][i].year != contract_year and df['trading_date'][i].month < expire_date.month:
            df.loc[i, 'tradeability'] = 'not to trade'
        else:
            df.loc[i, 'tradeability'] = 'tradeable'
    # INITIALIZE VARIABLES
    df['expire_date'] = expire_dates
    df['EMA20'] = ta.EMA(df['close'], timeperiod=20)
    df['EMA10'] = ta.EMA(df['close'], timeperiod=10)
    df['EMA5'] = ta.EMA(df['close'], timeperiod=5)
    df['ADX'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)
    df['BIAS'] = (df['close'] - df['close'].rolling(window=20).mean()) / df['close'].rolling(window=20).mean() * 100
    position = 0  
    initial_cash = 10000
    current_cash = initial_cash
    current_capital = initial_cash 
    trading_cost = 0  
    #total_trading_cost = 0
    action = None
    trades = []
    daily = []
    for i in range(1, len(df)):
        trading_date = df['trading_date'][i]
        if df['tradeability'][i] == 'tradeable':
            # LONG TREND FOLLWOING STRATEGY
            if contract_strategy['LONG/SHORT'] == 'LONG':
                if df['ADX'][i-1] > adx_optimal and \
                   df['close'][i-1] > df['EMA10'][i-1] and \
                   df['EMA10'][i-1] > df['EMA20'][i-1] and \
                   df['EMA10'][i-1] > df['EMA10'][i-2] and \
                   df['EMA20'][i-1] > df['EMA20'][i-2] and \
                   df['BIAS'][i-1] < bias_optimal and \
                   position < 3:
                    action = 'BUY'
                    price = df['open'][i]
                    position += 1
                    trading_cost = 0.0005 * price 
                    #total_trading_cost += trading_cost
                    current_cash -= (price + trading_cost)
                    current_capital = current_cash + (position * df['open'][i])
                    trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                elif position > 0:
                    if df['close'][i-1] < df['EMA10'][i-1] or \
                       df['BIAS'][i-1] > bias_optimal:
                        action = 'SELL'
                        price = df['open'][i]
                        current_cash += price * position
                        trading_cost = 0.0005 * price * position
                        position = 0
                        #total_trading_cost += trading_cost
                        current_cash -= trading_cost
                        current_capital = current_cash
                        trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                    else:
                        action = 'HOLD'
                        price = None
                        trading_cost = 0
                        if df['close'][i] > df['close'][i-1] or \
                           df['close'][i] == df['close'][i-1]:
                            current_cash += position * (df['close'][i] - df['close'][i-1])
                        else:
                            current_cash -= position * (df['close'][i-1] - df['close'][i])
                        current_capital = current_cash + (position * df['open'][i])
                else:
                    action = None
                    #price = None
                    trading_cost = 0
                    #current_cash = current_cash
            # SHORT TREND FOLLWOING STRATEGY
            if contract_strategy['LONG/SHORT'] == 'SHORT':
                if df['ADX'][i-1] > adx_optimal and \
                   df['close'][i-1] < df['EMA10'][i-1] and \
                   df['EMA10'][i-1] < df['EMA20'][i-1] and \
                   df['EMA10'][i-1] < df['EMA10'][i-2] and \
                   df['EMA20'][i-1] < df['EMA20'][i-2] and \
                   df['BIAS'][i-1] > bias_optimal and \
                   position < 1:
                    action = 'SELL'
                    price = df['open'][i]
                    position += 1
                    trading_cost = 0.0005 * price 
                    #total_trading_cost += trading_cost
                    current_cash -= (price + trading_cost)
                    current_capital = current_cash + (position * df['open'][i])
                    trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                elif position > 0:
                    if df['close'][i-1] > df['EMA10'][i-1] or \
                       df['BIAS'][i-1] < bias_optimal:
                        action = 'BUY'
                        price = df['open'][i]
                        current_cash += price * position
                        trading_cost = 0.0005 * price * position
                        position = 0
                        #total_trading_cost += trading_cost
                        current_cash -= trading_cost
                        current_capital = current_cash
                        trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                    else:
                        action = 'HOLD'
                        price = None
                        trading_cost = 0
                        if df['close'][i] > df['close'][i-1] or \
                           df['close'][i] == df['close'][i-1]:
                            current_cash += position * (df['close'][i] - df['close'][i-1])
                        else:
                            current_cash -= position * (df['close'][i-1] - df['close'][i])
                        current_capital = current_cash + (position * df['open'][i])
                else:
                    action = None
                    #price = None
                    trading_cost = 0
                    #current_cash = current_cash
        else:
            action = None
            #price = None
            trading_cost = 0
            #current_cash = current_cash
        daily_capital = current_cash + (position * df['close'][i])
        daily.append([trading_date, df['close'][i], action, df['open'][i], position, trading_cost, current_cash, current_capital, daily_capital]) 
        trades_df = pd.DataFrame(trades, columns=['trading_date', 'action', 'price', 'current_share_holding', 'trading_cost', 'current_cash', 'current_capital'])
        daily_df = pd.DataFrame(daily, columns=['date', 'close', 'action', 'open', 'current_share_holding', 'trading_cost', 'current_cash', 'current_capital', 'daily_capital'])
    # PERFORMANCE METRICS PER TRADE AND PERFORMANCE METRICS DAILY
    trades_df['per_trade_returns'] = trades_df['current_capital'].pct_change()
    daily_df['daily_returns'] = daily_df['daily_capital'].pct_change()
    sharpe_ratio_per_trade = trades_df['per_trade_returns'].mean() / trades_df['per_trade_returns'].std()
    sharpe_ratio_daily = daily_df['daily_returns'].mean() / daily_df['daily_returns'].std()
    max_drawdown_per_trade = (1 - (trades_df['current_capital'] / trades_df['current_capital'].cummax())).max()
    max_drawdown_daily = (1 - (daily_df['daily_capital'] / daily_df['daily_capital'].cummax())).max()
    per_trade_returns = trades_df['per_trade_returns']
    def calculate_gain_to_pain_ratio_per_trade(per_trade_returns):
        total_pain_per_trade = sum([r if r < 0 else 0 for r in per_trade_returns])
        total_gain_per_trade = sum([r if r > 0 else 0 for r in per_trade_returns])
        if total_pain_per_trade == 0:
            return float('inf')  
        else:
            return abs(total_gain_per_trade / total_pain_per_trade)
    gain_to_pain_ratio_per_trade = calculate_gain_to_pain_ratio_per_trade(per_trade_returns)
    daily_returns = daily_df['daily_returns']
    def calculate_gain_to_pain_ratio_daily(daily_returns): 
        total_pain_daily = sum([r if r < 0 else 0 for r in daily_returns])
        total_gain_daily = sum([r if r > 0 else 0 for r in daily_returns])
        if total_pain_daily == 0:
            return float('inf') 
        else:
            return abs(total_gain_daily / total_pain_daily)
    gain_to_pain_ratio_daily = calculate_gain_to_pain_ratio_daily(daily_returns)
    trades_df['sharpe_ratio_per_trade'] = sharpe_ratio_per_trade
    trades_df['max_drawdown_per_trade'] = max_drawdown_per_trade
    trades_df['gain_to_pain_ratio_per_trade'] = gain_to_pain_ratio_per_trade
    daily_df['sharpe_ratio_daily'] = sharpe_ratio_daily
    daily_df['max_drawdown_daily'] = max_drawdown_daily
    daily_df['gain_to_pain_ratio_daily'] = gain_to_pain_ratio_daily
    # OUTPUT TRADE RECORDS AND DAILY PNLS ALONG WITH PERFORMANCE METRICS to "_trades.xlsx" and "_daily.xlsx"
    result_folder = "CTA_result"
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)
    trades_excel_path = os.path.join(result_folder, os.path.basename(file_path).replace(".csv", "_trades.xlsx"))
    trades_df.to_excel(trades_excel_path, index=False, float_format='%.4f')
    daily_excel_path = os.path.join(result_folder, os.path.basename(file_path).replace(".csv", "_daily.xlsx"))
    daily_df.to_excel(daily_excel_path, index=False, float_format='%.4f')
    print(f"Processed {file_path} and saved trade record to {result_folder}")
    print("Sharpe Ratio Per Trade:", sharpe_ratio_per_trade)
    print("Max Drawdown Per Trade:", max_drawdown_per_trade)
    print("Gain-to-Pain Ratio Per Trade:", gain_to_pain_ratio_per_trade)
    print(f"Processed {file_path} and saved daily record to {result_folder}")
    print("Sharpe Ratio Daily:", sharpe_ratio_daily)
    print("Max Drawdown Daily:", max_drawdown_daily)
    print("Gain-to-Pain Ratio Daily:", gain_to_pain_ratio_daily)
    # OUTPUT PERFORMANCE METRICS PER TRADE FOR ALL CONTRACTS TO "performance_metrics_per_trade.csv"
    performance_metrics_per_trade_df = pd.DataFrame({
        'Code': [os.path.basename(file_path).replace(".csv", "")],
        'Sharpe_Ratio_Per_Trade': [sharpe_ratio_per_trade],
        'Max_Drawdown_Per_Trade': [max_drawdown_per_trade],
        'Gain_to_Pain_Ratio_Per_Trade': [gain_to_pain_ratio_per_trade]
    })
    performance_metrics_per_trade_file = os.path.join(result_folder, "performance_metrics_per_trade.csv")
    if not os.path.exists(performance_metrics_per_trade_file):
        performance_metrics_per_trade_df.to_csv(performance_metrics_per_trade_file, index=False)
    else:
        performance_metrics_per_trade_df.to_csv(performance_metrics_per_trade_file, mode='a', header=False, index=False)
    print(f"Saved performance metrics per trade to {performance_metrics_per_trade_file}")
    # OUTPUT PERFORMANCE METRICS DAILY FOR ALL CONTRACTS TO "performance_metrics_daily.csv"
    performance_metrics_daily_df = pd.DataFrame({
        'Code': [os.path.basename(file_path).replace(".csv", "")],
        'Sharpe_Ratio_Daily': [sharpe_ratio_daily],
        'Max_Drawdown_Daily': [max_drawdown_daily],
        'Gain_to_Pain_Ratio_Daily': [gain_to_pain_ratio_daily]
    })
    performance_metrics_daily_file = os.path.join(result_folder, "performance_metrics_daily.csv")
    if not os.path.exists(performance_metrics_daily_file):
        performance_metrics_daily_df.to_csv(performance_metrics_daily_file, index=False)
    else:
        performance_metrics_daily_df.to_csv(performance_metrics_daily_file, mode='a', header=False, index=False)
    print(f"Saved performance metrics daily to {performance_metrics_daily_file}")
if __name__ == "__main__":
    input_folder = "C:/Users/Jiaxu/Desktop/quant/CTA_data"
    result_folder = "C:/Users/Jiaxu/Desktop/quant/CTA_result"
    strategy_file = "C:/Users/Jiaxu/Desktop/quant/CTA_data/instruction/CTA_strategy.csv"
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)
    strategy_df = read_strategy(strategy_file)
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_folder, filename)
            process_csv(file_path, strategy_df)

In [ ]:
# CTA_ver2: IMPLEMENT CTA STRATEGY WITH OPTIMAL (ADX, BIAS) PAIRS FOR ALL CONTRACTS TO GENERATE TRADE RECORDS, DAILY PNLS, AND PERFORMANCE METRICS (May-20)
# coding="utf-8"
import os
import sys
sys.path.insert(0, os.getcwd())
import talib as ta
import pandas as pd
import numpy as np
import datetime
# READ STRATEGY FILE
def read_strategy(strategy_file):
    strategy_df = pd.read_csv(strategy_file)
    return strategy_df
# PROCESS HISTROICAL DATA FILES
def process_csv(file_path, strategy_df):
    # EXTRACT CTA STRATEGY INFORMATION FROM quant/CTA_data/instruction/CTA_strategy.csv
    contract_code = os.path.basename(file_path).replace(".csv", "")
    print("Contract Code:", contract_code)
    contract_strategies = strategy_df[strategy_df['CODE'] == contract_code]
    for index, contract_strategy in contract_strategies.iterrows():
        print(contract_strategy)
        if contract_strategy['RUN_OR_NOT'] != 'RUN':
            print(f"Skipping {file_path} as per strategy instruction.")
            continue
        adx_optimal = contract_strategy['ADX_OPTIMAL']
        bias_optimal = contract_strategy['BIAS_OPTIMAL']
        # LOAD DATA AND DECIDE TRADEABILITY
        df = pd.read_csv(file_path, engine='python')
        df.columns = df.columns.str.replace(' ', '')
        df.fillna(value={'open': df['close']}, inplace=True)
        df.interpolate(method='nearest', inplace=True)
        df = df.dropna(axis=1, how='all')
        df = df.iloc[::-1]
        df = df.reset_index(drop=True)
        df['trading_date'] = pd.to_datetime(df['trading_date'], format='%m/%d/%Y')
        df['contract_year'] = df['contract_year'].apply(lambda x: int('20' + x.split('^')[1] + x.split('^')[0]))
        if 'N' in df['contract_month'].values:
            month_code = 'N'
        elif 'V' in df['contract_month'].values:
            month_code = 'V'
        elif 'X' in df['contract_month'].values:
            month_code = 'X'
        elif 'Z' in df['contract_month'].values:
            month_code = 'Z'
        else:
            month_code = input("Enter the month code of the contract (N, V, X, Z): ")
        expire_dates = []
        for i in range(len(df)):
            contract_year = df['contract_year'][i]
            if month_code == 'N':
                expire_date = pd.Timestamp(str(contract_year) + '-07-01')
            elif month_code == 'V':
                expire_date = pd.Timestamp(str(contract_year) + '-10-01')
            elif month_code == 'X':
                expire_date = pd.Timestamp(str(contract_year) + '-11-01')
            elif month_code == 'Z':
                expire_date = pd.Timestamp(str(contract_year) + '-12-01')
            else:
                print("Invalid month code!")
                break
            expire_dates.append(expire_date)
            if df['trading_date'][i].year == contract_year:
                if df['trading_date'][i] > expire_date:
                    df.loc[i, 'tradeability'] = 'nontradeable'
                else:
                    df.loc[i, 'tradeability'] = 'tradeable'
            elif df['trading_date'][i].year != contract_year and df['trading_date'][i].month < expire_date.month:
                df.loc[i, 'tradeability'] = 'not to trade'
            else:
                df.loc[i, 'tradeability'] = 'tradeable'
        # INITIALIZE VARIABLES
        df['expire_date'] = expire_dates
        df['EMA20'] = ta.EMA(df['close'], timeperiod=20)
        df['EMA10'] = ta.EMA(df['close'], timeperiod=10)
        df['EMA5'] = ta.EMA(df['close'], timeperiod=5)
        df['ADX'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)
        df['BIAS'] = (df['close'] - df['close'].rolling(window=20).mean()) / df['close'].rolling(window=20).mean() * 100
        position = 0  
        initial_cash = 10000
        current_cash = initial_cash
        current_capital = initial_cash 
        trading_cost = 0  
        total_trading_cost = 0
        action = None
        trades = []
        daily = []
        for i in range(1, len(df)):
            trading_date = df['trading_date'][i]
            if df['tradeability'][i] == 'tradeable':
                # LONG TREND FOLLWOING STRATEGY
                if contract_strategy['LONG/SHORT'] == 'LONG':
                    if df['ADX'][i-1] > adx_optimal and \
                       df['close'][i-1] > df['EMA10'][i-1] and \
                       df['EMA10'][i-1] > df['EMA20'][i-1] and \
                       df['EMA10'][i-1] > df['EMA10'][i-2] and \
                       df['EMA20'][i-1] > df['EMA20'][i-2] and \
                       df['BIAS'][i-1] < bias_optimal and \
                       position < 3:
                        action = 'BUY'
                        price = df['open'][i]
                        position += 1
                        trading_cost = 0.0005 * price 
                        total_trading_cost += trading_cost
                        current_cash -= (price + trading_cost)
                        current_capital = current_cash + (position * df['open'][i])
                        trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                    elif position > 0:
                        if df['close'][i-1] < df['EMA10'][i-1] or \
                           df['BIAS'][i-1] > bias_optimal:
                            action = 'SELL'
                            price = df['open'][i]
                            current_cash += price * position
                            trading_cost = 0.0005 * price * position
                            position = 0
                            total_trading_cost += trading_cost
                            current_cash -= trading_cost
                            current_capital = current_cash
                            trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                        else:
                            action = 'HOLD'
                            price = None
                            trading_cost = 0
                            if df['close'][i] > df['close'][i-1] or \
                               df['close'][i] == df['close'][i-1]:
                                current_cash += position * (df['close'][i] - df['close'][i-1])
                                current_capital += position * (df['close'][i] - df['close'][i-1])
                            else:
                                current_cash -= position * (df['close'][i-1] - df['close'][i])
                                current_capital -= position * (df['close'][i-1] - df['close'][i])
                            #trades.append([trading_date, action, price, position, trading_cost, current_capital]) 
                    else:
                        action = None
                        #price = None
                        trading_cost = 0
                        #current_cash = current_cash
                # SHORT TREND FOLLWOING STRATEGY
                if contract_strategy['LONG/SHORT'] == 'SHORT':
                    if df['ADX'][i-1] > adx_optimal and \
                       df['close'][i-1] < df['EMA10'][i-1] and \
                       df['EMA10'][i-1] < df['EMA20'][i-1] and \
                       df['EMA10'][i-1] < df['EMA10'][i-2] and \
                       df['EMA20'][i-1] < df['EMA20'][i-2] and \
                       df['BIAS'][i-1] > bias_optimal and \
                       position < 1:
                        action = 'SELL'
                        price = df['open'][i]
                        position += 1
                        trading_cost = 0.0005 * price 
                        total_trading_cost += trading_cost
                        current_cash -= (price + trading_cost)
                        current_capital = current_cash + (position * df['open'][i])
                        trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                    elif position > 0:
                        if df['close'][i-1] > df['EMA10'][i-1] or \
                           df['BIAS'][i-1] < bias_optimal:
                            action = 'BUY'
                            price = df['open'][i]
                            current_cash += price * position
                            trading_cost = 0.0005 * price * position
                            position = 0
                            total_trading_cost += trading_cost
                            current_cash -= trading_cost
                            current_capital = current_cash
                            trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                        else:
                            action = 'HOLD'
                            price = None
                            trading_cost = 0
                            if df['close'][i] < df['close'][i-1] or \
                               df['close'][i] == df['close'][i-1]:
                                current_cash += position * (df['close'][i-1] - df['close'][i])
                                current_capital += position * (df['close'][i-1] - df['close'][i])
                            else:
                                current_cash -= position * (df['close'][i] - df['close'][i-1])
                                current_capital -= position * (df['close'][i] - df['close'][i-1])
                            #trades.append([trading_date, action, price, position, trading_cost, current_capital]) 
                    else:
                        action = None
                        #price = None
                        trading_cost = 0
                        #current_cash = current_cash
            else:
                action = None
                #price = None
                trading_cost = 0
                #current_cash = current_cash
            daily_capital = current_cash + (position * df['close'][i])
            daily.append([trading_date, df['close'][i], action, df['open'][i], position, trading_cost, current_cash, current_capital, daily_capital]) 
            #check indent position!!!
            trades_df = pd.DataFrame(trades, columns=['trading_date', 'action', 'price', 'current_share_holding', 'trading_cost', 'current_cash', 'current_capital'])
            daily_df = pd.DataFrame(daily, columns=['date', 'close', 'action', 'open', 'current_share_holding', 'trading_cost', 'current_cash', 'current_capital', 'daily_capital'])
        # PERFORMANCE METRICS PER TRADE AND PERFORMANCE METRICS DAILY
        trades_df['per_trade_returns'] = trades_df['current_capital'].pct_change()
        daily_df['daily_returns'] = daily_df['daily_capital'].pct_change()
        sharpe_ratio_per_trade = trades_df['per_trade_returns'].mean() / trades_df['per_trade_returns'].std()
        sharpe_ratio_daily = daily_df['daily_returns'].mean() / daily_df['daily_returns'].std()
        max_drawdown_per_trade = (1 - (trades_df['current_capital'] / trades_df['current_capital'].cummax())).max()
        max_drawdown_daily = (1 - (daily_df['daily_capital'] / daily_df['daily_capital'].cummax())).max()
        per_trade_returns = trades_df['per_trade_returns']
        def calculate_gain_to_pain_ratio_per_trade(per_trade_returns):
            total_pain_per_trade = sum([r if r < 0 else 0 for r in per_trade_returns])
            total_gain_per_trade = sum([r if r > 0 else 0 for r in per_trade_returns])
            if total_pain_per_trade == 0:
                return float('inf')  
            else:
                return abs(total_gain_per_trade / total_pain_per_trade)
        gain_to_pain_ratio_per_trade = calculate_gain_to_pain_ratio_per_trade(per_trade_returns)
        daily_returns = daily_df['daily_returns']
        def calculate_gain_to_pain_ratio_daily(daily_returns): 
            total_pain_daily = sum([r if r < 0 else 0 for r in daily_returns])
            total_gain_daily = sum([r if r > 0 else 0 for r in daily_returns])
            if total_pain_daily == 0:
                return float('inf') 
            else:
                return abs(total_gain_daily / total_pain_daily)
        gain_to_pain_ratio_daily = calculate_gain_to_pain_ratio_daily(daily_returns)
        #final_profit_per_trade = (trades_df['current_capital'].iloc[-1] - initial_cash) / df['close'].mean()
        final_profit = (daily_df['daily_capital'].iloc[-1] - initial_cash) / df['close'].mean()
        trades_df['sharpe_ratio_per_trade'] = sharpe_ratio_per_trade
        trades_df['max_drawdown_per_trade'] = max_drawdown_per_trade
        trades_df['gain_to_pain_ratio_per_trade'] = gain_to_pain_ratio_per_trade
        #trades_df['final_profit_per_trade'] = final_profit_per_trade
        daily_df['sharpe_ratio_daily'] = sharpe_ratio_daily
        daily_df['max_drawdown_daily'] = max_drawdown_daily
        daily_df['gain_to_pain_ratio_daily'] = gain_to_pain_ratio_daily
        daily_df['final_profit'] = final_profit

        # OUTPUT TRADE RECORDS AND DAILY PNLS ALONG WITH PERFORMANCE METRICS to "_trades.xlsx" and "_daily.xlsx"
        result_folder = "CTA_result"
        if not os.path.exists(result_folder):
            os.makedirs(result_folder)
        # OUTPUT FILE PATHS WITH SUFFIX DENOTING LONG OR SHORT STRATEGY
        output_suffix = '_long' if contract_strategy['LONG/SHORT'] == 'LONG' else '_short'
        trades_excel_path = os.path.join(result_folder, f"{os.path.basename(file_path).replace('.csv', '')}{output_suffix}_trades.xlsx")
        daily_excel_path = os.path.join(result_folder, f"{os.path.basename(file_path).replace('.csv', '')}{output_suffix}_daily.xlsx")
        trades_df.to_excel(trades_excel_path, index=False, float_format='%.4f')
        daily_df.to_excel(daily_excel_path, index=False, float_format='%.4f')
        print(f"Processed {file_path} and saved trade record to {trades_excel_path}")
        print("Sharpe Ratio Per Trade:", sharpe_ratio_per_trade)
        print("Max Drawdown Per Trade:", max_drawdown_per_trade)
        print("Gain-to-Pain Ratio Per Trade:", gain_to_pain_ratio_per_trade)
        #print("Final Profit Per Trade:", final_profit_per_trade)
        print(f"Processed {file_path} and saved daily record to {daily_excel_path}")
        print("Sharpe Ratio Daily:", sharpe_ratio_daily)
        print("Max Drawdown Daily:", max_drawdown_daily)
        print("Gain-to-Pain Ratio Daily:", gain_to_pain_ratio_daily)
        print("Final Profit:", final_profit)
        # SAVE PERFORMANCE METRICS PER TRADE
        performance_metrics_per_trade_df = pd.DataFrame({
            'Code': [os.path.basename(file_path).replace(".csv", "")],
            'Strategy': [contract_strategy['LONG/SHORT']],
            'Sharpe_Ratio_Per_Trade': [sharpe_ratio_per_trade],
            'Max_Drawdown_Per_Trade': [max_drawdown_per_trade],
            'Gain_to_Pain_Ratio_Per_Trade': [gain_to_pain_ratio_per_trade],
            #'Final_Profit_Per_Trade': [final_profit_per_trade]
        })
        performance_metrics_per_trade_file = os.path.join(result_folder, "performance_metrics_per_trade.csv")
        if not os.path.exists(performance_metrics_per_trade_file):
            performance_metrics_per_trade_df.to_csv(performance_metrics_per_trade_file, index=False)
        else:
            performance_metrics_per_trade_df.to_csv(performance_metrics_per_trade_file, mode='a', header=False, index=False)
        print(f"Saved performance metrics per trade to {performance_metrics_per_trade_file}")
        # SAVE PERFORMANCE METRICS DAILY
        performance_metrics_daily_df = pd.DataFrame({
            'Code': [os.path.basename(file_path).replace(".csv", "")],
            'Strategy': [contract_strategy['LONG/SHORT']],
            'Sharpe_Ratio_Daily': [sharpe_ratio_daily],
            'Max_Drawdown_Daily': [max_drawdown_daily],
            'Gain_to_Pain_Ratio_Daily': [gain_to_pain_ratio_daily],
            'Final_Profit': [final_profit]
        })
        performance_metrics_daily_file = os.path.join(result_folder, "performance_metrics_daily.csv")
        if not os.path.exists(performance_metrics_daily_file):
            performance_metrics_daily_df.to_csv(performance_metrics_daily_file, index=False)
        else:
            performance_metrics_daily_df.to_csv(performance_metrics_daily_file, mode='a', header=False, index=False)
        print(f"Saved performance metrics daily to {performance_metrics_daily_file}")
if __name__ == "__main__":
    input_folder = "C:/Users/Jiaxu/Desktop/quant/CTA_data"
    result_folder = "C:/Users/Jiaxu/Desktop/quant/CTA_result"
    strategy_file = "C:/Users/Jiaxu/Desktop/quant/CTA_data/instruction/CTA_strategy.csv"
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)
    strategy_df = read_strategy(strategy_file)
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_folder, filename)
            process_csv(file_path, strategy_df)

In [ ]:
# CTA_ver3: IMPLEMENT CTA STRATEGY FOR ALL CONTRACTS IN MONETARY BASIS (THEORETICALLY ENOUGH INITIAL CAPITAL)
# coding="utf-8"
import os
import sys
sys.path.insert(0, os.getcwd())
import talib as ta
import pandas as pd
import numpy as np
import datetime
import openpyxl
import csv  
import io
import re

# READ STRATEGY FILE
def read_strategy(strategy_file):
    strategy_df = pd.read_csv(strategy_file)
    return strategy_df

# FUNCTION TO HANDLE FILE READING AND PARSING
def read_csv_file(file_path):
    try:
        # READ THE FILE IN BINARY MODE, REMOVE NULL BYTES, AND DECODE
        with open(file_path, 'rb') as f:
            content = f.read().replace(b'\x00', b'')
        decoded_content = content.decode('latin1')
        # USE io.StringIO TO CREATE A FILE-LIKE OBJECT FOR PANDAS TO READ
        df = pd.read_csv(io.StringIO(decoded_content), encoding='utf-8', engine='c')
        print(f"Successfully read {file_path} after preprocessing.")
        return df
    except Exception as e:
        print(f"Failed to read {file_path} after preprocessing: {e}")
        return None

# PROCESS HISTORICAL DATA FILES
def process_csv(file_path, strategy_df):
    # EXTRACT CTA STRATEGY INFORMATION FROM quant/CTA_data/instruction/CTA_strategy.csv
    contract_code = os.path.basename(file_path).replace(".csv", "")
    print("Contract Code:", contract_code)
    contract_strategies = strategy_df[strategy_df['CODE'] == contract_code]
    for index, contract_strategy in contract_strategies.iterrows():
        print(contract_strategy)
        if contract_strategy['RUN_OR_NOT'] != 'RUN':
            print(f"Skipping {file_path} strategy instruction.")
            continue
        adx_optimal = contract_strategy['ADX_OPTIMAL']
        bias_optimal = contract_strategy['BIAS_OPTIMAL']
        multiplier = contract_strategy['MULTIPLIER']
        # LOAD DATA AND DECIDE TRADEABILITY
        historical_data_file = os.path.join("CTA_data", f"{contract_code}.csv")
        df = read_csv_file(historical_data_file)
        if df is None:
            continue
        df.columns = df.columns.str.replace(' ', '')
        df.fillna(value={'open': df['close']}, inplace=True)
        df.interpolate(method='nearest', inplace=True)
        df = df.dropna(axis=1, how='all')
        df = df.iloc[::-1]
        df = df.reset_index(drop=True)
        df['trading_date'] = pd.to_datetime(df['trading_date'], format='%m/%d/%Y')
        df['contract_year'] = df['contract_year'].apply(lambda x: int('20' + x.split('^')[1] + x.split('^')[0]))
        if 'N' in df['contract_month'].values:
            month_code = 'N'
        elif 'V' in df['contract_month'].values:
            month_code = 'V'
        elif 'X' in df['contract_month'].values:
            month_code = 'X'
        elif 'Z' in df['contract_month'].values:
            month_code = 'Z'
        else:
            month_code = input("Enter the month code of the contract (N, V, X, Z): ")
        expire_dates = []
        for i in range(len(df)):
            contract_year = df['contract_year'][i]
            if month_code == 'N':
                expire_date = pd.Timestamp(str(contract_year) + '-07-01')
            elif month_code == 'V':
                expire_date = pd.Timestamp(str(contract_year) + '-10-01')
            elif month_code == 'X':
                expire_date = pd.Timestamp(str(contract_year) + '-11-01')
            elif month_code == 'Z':
                expire_date = pd.Timestamp(str(contract_year) + '-12-01')
            else:
                print("Invalid month code!")
                break
            expire_dates.append(expire_date)
            if df['trading_date'][i].year == contract_year:
                if df['trading_date'][i] > expire_date or df['trading_date'][i] == expire_date:
                    df.loc[i, 'tradeability'] = 'nontradeable'
                else:
                    df.loc[i, 'tradeability'] = 'tradeable'
            elif df['trading_date'][i].year != contract_year and df['trading_date'][i].month < expire_date.month:
                df.loc[i, 'tradeability'] = 'not to trade'
            else:
                df.loc[i, 'tradeability'] = 'tradeable'
        # INITIALIZE VARIABLES
        df['expire_date'] = expire_dates
        df['EMA20'] = ta.EMA(df['close'], timeperiod=20)
        df['EMA10'] = ta.EMA(df['close'], timeperiod=10)
        df['EMA5'] = ta.EMA(df['close'], timeperiod=5)
        df['ADX'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)
        df['BIAS'] = (df['close'] - df['close'].rolling(window=20).mean()) / df['close'].rolling(window=20).mean() * 100
        position = 0  
        initial_cash = 10000 # not exactly initial cash, but enough points to cover the margin (transform to monetary basis later)
        current_cash = initial_cash
        current_capital = initial_cash 
        trading_cost = 0  
        total_trading_cost = 0
        action = None
        trades = []
        daily = []
        for i in range(1, len(df)):
            trading_date = df['trading_date'][i]
            if df['tradeability'][i] == 'tradeable':
                # LONG TREND FOLLOWING STRATEGY
                if contract_strategy['LONG/SHORT'] == 'LONG':
                    if df['ADX'][i-1] > adx_optimal and \
                       df['close'][i-1] > df['EMA10'][i-1] and \
                       df['EMA10'][i-1] > df['EMA20'][i-1] and \
                       df['EMA10'][i-1] > df['EMA10'][i-2] and \
                       df['EMA20'][i-1] > df['EMA20'][i-2] and \
                       df['BIAS'][i-1] < bias_optimal and \
                       position < 3:
                        action = 'BUY'
                        price = df['open'][i]
                        position += 1
                        trading_cost = 0.0005 * price 
                        total_trading_cost += trading_cost
                        current_cash -= (price + trading_cost)
                        current_capital = current_cash + (position * df['open'][i])
                        trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                    elif position > 0:
                        if df['close'][i-1] < df['EMA10'][i-1] or \
                           df['BIAS'][i-1] > bias_optimal:
                            action = 'SELL'
                            price = df['open'][i]
                            current_cash += price * position
                            trading_cost = 0.0005 * price * position
                            position = 0
                            total_trading_cost += trading_cost
                            current_cash -= trading_cost
                            current_capital = current_cash
                            trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                        else:
                            action = 'HOLD'
                            price = None
                            trading_cost = 0
                            if df['close'][i] > df['close'][i-1] or \
                               df['close'][i] == df['close'][i-1]:
                                current_cash += position * (df['close'][i] - df['close'][i-1])
                                current_capital += position  * (df['close'][i] - df['close'][i-1])
                            else:
                                current_cash -= position * (df['close'][i-1] - df['close'][i])
                                current_capital -= position * (df['close'][i-1] - df['close'][i])
                    else:
                        action = None
                        trading_cost = 0
                # SHORT TREND FOLLOWING STRATEGY
                if contract_strategy['LONG/SHORT'] == 'SHORT':
                    if df['ADX'][i-1] > adx_optimal and \
                       df['close'][i-1] < df['EMA10'][i-1] and \
                       df['EMA10'][i-1] < df['EMA20'][i-1] and \
                       df['EMA10'][i-1] < df['EMA10'][i-2] and \
                       df['EMA20'][i-1] < df['EMA20'][i-2] and \
                       df['BIAS'][i-1] > bias_optimal and \
                       position < 1:
                        action = 'SELL'
                        price = df['open'][i]
                        position += 1
                        trading_cost = 0.0005 * price 
                        total_trading_cost += trading_cost
                        current_cash -= (price + trading_cost)
                        current_capital = current_cash + (position * df['open'][i])
                        trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                    elif position > 0:
                        if df['close'][i-1] > df['EMA10'][i-1] or \
                           df['BIAS'][i-1] < bias_optimal:
                            action = 'BUY'
                            price = df['open'][i]
                            current_cash += price * position
                            trading_cost = 0.0005 * price * position
                            position = 0
                            total_trading_cost += trading_cost
                            current_cash -= trading_cost
                            current_capital = current_cash
                            trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                        else:
                            action = 'HOLD'
                            price = None
                            trading_cost = 0
                            if df['close'][i] < df['close'][i-1] or \
                               df['close'][i] == df['close'][i-1]:
                                current_cash += position * (df['close'][i-1] - df['close'][i])
                                current_capital += position * (df['close'][i-1] - df['close'][i])
                            else:
                                current_cash -= position * (df['close'][i] - df['close'][i-1])
                                current_capital -= position * (df['close'][i] - df['close'][i-1])
                    else:
                        action = None
                        trading_cost = 0
            #else:
                #action = None
                #trading_cost = 0
                daily_capital = current_cash + (position * df['close'][i])
                daily.append([trading_date, df['close'][i], action, df['open'][i], position, trading_cost, current_cash, current_capital, daily_capital]) 
            #check indent position!!!
            trades_df = pd.DataFrame(trades, columns=['trading_date', 'action', 'price', 'current_share_holding', 'trading_cost', 'current_cash', 'current_capital'])
            daily_df = pd.DataFrame(daily, columns=['date', 'close', 'action', 'open', 'current_share_holding', 'trading_cost', 'current_cash', 'current_capital', 'daily_capital'])
        
        # PERFORMANCE METRICS PER TRADE AND PERFORMANCE METRICS DAILY
        # TRANSFORM TO MONETARY BASIS
        daily_df['daily_returns_diff'] = daily_df['daily_capital'] - daily_df['daily_capital'].shift(1)
        daily_df['daily_point'] = daily_df['current_capital'] - 10000 
        daily_point = daily_df['daily_point']
        monetary_daily_profit = daily_point * multiplier
        daily_df['monetary_daily_profit'] = monetary_daily_profit
        
        trades_df['per_trade_returns_pct_change'] = trades_df['current_capital'].pct_change()
        daily_df['daily_returns_pct_change'] = daily_df['daily_capital'].pct_change()

        sharpe_ratio_per_trade = trades_df['per_trade_returns_pct_change'].mean() / trades_df['per_trade_returns_pct_change'].std()
        sharpe_ratio_daily = daily_df['daily_returns_pct_change'].mean() / daily_df['daily_returns_pct_change'].std()

        max_drawdown_per_trade = (1 - (trades_df['current_capital'] / trades_df['current_capital'].cummax())).max()
        max_drawdown_daily = (1 - (daily_df['daily_capital'] / daily_df['daily_capital'].cummax())).max()

        per_trade_returns_pct_change = trades_df['current_capital'].pct_change()
        def calculate_gain_to_pain_ratio_per_trade(per_trade_returns_pct_change):
            total_pain_per_trade = sum([r if r < 0 else 0 for r in per_trade_returns_pct_change])
            total_gain_per_trade = sum([r if r > 0 else 0 for r in per_trade_returns_pct_change])
            if total_pain_per_trade == 0:
                return float('inf')  
            else:
                return abs(total_gain_per_trade / total_pain_per_trade)
        gain_to_pain_ratio_per_trade = calculate_gain_to_pain_ratio_per_trade(per_trade_returns_pct_change)
       
        daily_returns_pct_change = daily_df['daily_capital'].pct_change()
        def calculate_gain_to_pain_ratio_daily(daily_returns_pct_change): 
            total_pain_daily = sum([r if r < 0 else 0 for r in daily_returns_pct_change])
            total_gain_daily = sum([r if r > 0 else 0 for r in daily_returns_pct_change])
            if total_pain_daily == 0:
                return float('inf') 
            else:
                return abs(total_gain_daily / total_pain_daily)
        gain_to_pain_ratio_daily = calculate_gain_to_pain_ratio_daily(daily_returns_pct_change)

        final_profit = (daily_df['daily_capital'].iloc[-1] - initial_cash) / df['close'].mean()
        trades_df['sharpe_ratio_per_trade'] = sharpe_ratio_per_trade
        trades_df['max_drawdown_per_trade'] = max_drawdown_per_trade
        trades_df['gain_to_pain_ratio_per_trade'] = gain_to_pain_ratio_per_trade
        
        daily_df['sharpe_ratio_daily'] = sharpe_ratio_daily
        daily_df['max_drawdown_daily'] = max_drawdown_daily
        daily_df['gain_to_pain_ratio_daily'] = gain_to_pain_ratio_daily
        daily_df['final_profit'] = final_profit
        
        # OUTPUT TRADE RECORDS AND DAILY PNLS ALONG WITH PERFORMANCE METRICS to "_trades.xlsx" and "_daily.xlsx"
        result_folder = "CTA_result"
        if not os.path.exists(result_folder):
            os.makedirs(result_folder)
        # OUTPUT FILE PATHS WITH SUFFIX DENOTING LONG OR SHORT STRATEGY
        output_suffix = '_long' if contract_strategy['LONG/SHORT'] == 'LONG' else '_short'
        trades_output_file = os.path.join(result_folder, f"{contract_code}{output_suffix}_trades.xlsx")
        daily_output_file = os.path.join(result_folder, f"{contract_code}{output_suffix}_daily.xlsx")
        with pd.ExcelWriter(trades_output_file, engine='openpyxl') as writer:
            trades_df.to_excel(writer, sheet_name='trades', index=False, float_format='%.4f')
        with pd.ExcelWriter(daily_output_file, engine='openpyxl') as writer:
            daily_df.to_excel(writer, sheet_name='daily', index=False, float_format='%.4f')
    
        print(f"Processed {file_path} and saved trade record to {trades_output_file}")
        print("Sharpe Ratio Per Trade:", sharpe_ratio_per_trade)
        print("Max Drawdown Per Trade:", max_drawdown_per_trade)
        print("Gain-to-Pain Ratio Per Trade:", gain_to_pain_ratio_per_trade)
        
        print(f"Processed {file_path} and saved daily record to {daily_output_file}")
        print("Sharpe Ratio Daily:", sharpe_ratio_daily)
        print("Max Drawdown Daily:", max_drawdown_daily)
        print("Gain-to-Pain Ratio Daily:", gain_to_pain_ratio_daily)
        print("Final Profit:", final_profit)
    
        # AGGREGATE PROFITS AND LOSSES FROM ALL CONTRACTS FOR TOTAL DAILY PNL
        aggregated_profits = {}
        # ITERATE OVER ALL FILES IN THE CTA_result FOLDER
        for filename in os.listdir(result_folder):
            if filename.endswith("_daily.xlsx"): 
                file_path = os.path.join(result_folder, filename)
                contract_code = filename.split("_")[0] 
                daily_df = pd.read_excel(file_path, header=None, skiprows=1)
                dates = daily_df.iloc[:, 0] 
                profits = daily_df.iloc[:, 11] # extract monetary daily profit (12th column) from _daily.xlsx file
                actions = daily_df.iloc[:, 2]
                # ITERATE OVER EACH DATE, ACTION, AND PROFIT
                for date, action, profit in zip(dates, actions, profits):
                    if pd.notna(date):
                        if date in aggregated_profits:
                            existing_profit, existing_actions, contract_codes = aggregated_profits[date]
                        else:
                            existing_profit = 0
                            existing_actions = []
                            contract_codes = []
                        # SUM PROFIT FOR EACH DATE
                        new_profit = existing_profit + profit
                        existing_actions.append(action)
                        if action in ['BUY', 'SELL', 'HOLD']:
                            contract_codes.append(contract_code)
                        aggregated_profits[date] = (new_profit, existing_actions, contract_codes) # Update the aggregated_profits dictionary     
        aggregated_df = pd.DataFrame(list(aggregated_profits.items()), columns=['Date', 'Total Profit']) # Convert aggregated_profits dictionary to DataFrame
        # EXTRACT NUMERIC VALUE FROM 'Total Profit' USING REGULAR EXPRESSIONS
        def extract_numeric_value(text):
            matches = re.findall(r"[-+]?\d*\.\d+|\d+", str(text))  # Match decimal numbers
            if matches:
                return float(matches[0])  # Return the first match as float
            else:
                return None
        aggregated_df['Value'] = aggregated_df['Total Profit'].apply(extract_numeric_value)
        # SORT THE DATAFRAME BY DATE
        aggregated_df['Date'] = pd.to_datetime(aggregated_df['Date'], format='%Y-%m-%d') 
        aggregated_df = aggregated_df.dropna(subset=['Date'])  # Drop rows with invalid dates
        csv_file_path = os.path.join(result_folder, "monetary_daily_profit.csv")
        aggregated_df.to_csv(csv_file_path, index=False)
        print("Aggregated profits saved to:", csv_file_path)

def main():
    input_folder = "C:/Users/Jiaxu/Desktop/quant/CTA_data"
    result_folder = "C:/Users/Jiaxu/Desktop/quant/CTA_result"
    strategy_file = "C:/Users/Jiaxu/Desktop/quant/CTA_data/instruction/CTA_strategy.csv"
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)
    strategy_df = pd.read_csv(strategy_file)
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_folder, filename)
            process_csv(file_path, strategy_df)
if __name__ == "__main__":
    main()

In [ ]:
# CTA_ver33: IMPLEMENT CTA STRATEGY FOR ALL CONTRACTS IN MONETARY BASIS (THEORETICALLY ENOUGH INITIAL CAPITAL)
# coding="utf-8"
import os
import sys
sys.path.insert(0, os.getcwd())
import talib as ta
import pandas as pd
import numpy as np
import datetime
import openpyxl
import csv  
import io
import re

# READ STRATEGY FILE
def read_strategy(strategy_file):
    strategy_df = pd.read_csv(strategy_file)
    return strategy_df

# FUNCTION TO HANDLE FILE READING AND PARSING
def read_csv_file(file_path):
    try:
        # READ THE FILE IN BINARY MODE, REMOVE NULL BYTES, AND DECODE
        with open(file_path, 'rb') as f:
            content = f.read().replace(b'\x00', b'')
        decoded_content = content.decode('latin1')
        # USE io.StringIO TO CREATE A FILE-LIKE OBJECT FOR PANDAS TO READ
        df = pd.read_csv(io.StringIO(decoded_content), encoding='utf-8', engine='c')
        return df
    except Exception as e:
        print(f"Failed to read {file_path} after preprocessing: {e}")
        return None

# PROCESS HISTORICAL DATA FILES
def process_csv(file_path, strategy_df):
    # EXTRACT CTA STRATEGY INFORMATION FROM quant/CTA_data/instruction/CTA_strategy.csv
    contract_code = os.path.basename(file_path).replace(".csv", "")
    print("Contract Code:", contract_code)
    contract_strategies = strategy_df[strategy_df['CODE'] == contract_code]
    for index, contract_strategy in contract_strategies.iterrows():
        print(contract_strategy)
        if contract_strategy['RUN_OR_NOT'] != 'RUN':
            print(f"Skipping {file_path} as per strategy instruction.")
            continue
        adx_optimal = contract_strategy['ADX_OPTIMAL']
        bias_optimal = contract_strategy['BIAS_OPTIMAL']
        multiplier = contract_strategy['MULTIPLIER']
        # LOAD DATA AND DECIDE TRADEABILITY
        df = read_csv_file(file_path)
        if df is None:
            continue
        df.columns = df.columns.str.replace(' ', '')
        df.fillna(value={'open': df['close']}, inplace=True)
        df.interpolate(method='nearest', inplace=True)
        df = df.dropna(axis=1, how='all')
        df = df.iloc[::-1]
        df = df.reset_index(drop=True)
        df['trading_date'] = pd.to_datetime(df['trading_date'], format='%m/%d/%Y')
        df['contract_year'] = df['contract_year'].apply(lambda x: int('20' + x.split('^')[1] + x.split('^')[0]))
        if 'N' in df['contract_month'].values:
            month_code = 'N'
        elif 'V' in df['contract_month'].values:
            month_code = 'V'
        elif 'X' in df['contract_month'].values:
            month_code = 'X'
        elif 'Z' in df['contract_month'].values:
            month_code = 'Z'
        else:
            month_code = input("Enter the month code of the contract (N, V, X, Z): ")
        expire_dates = []
        for i in range(len(df)):
            contract_year = df['contract_year'][i]
            if month_code == 'N':
                expire_date = pd.Timestamp(str(contract_year) + '-07-01')
            elif month_code == 'V':
                expire_date = pd.Timestamp(str(contract_year) + '-10-01')
            elif month_code == 'X':
                expire_date = pd.Timestamp(str(contract_year) + '-11-01')
            elif month_code == 'Z':
                expire_date = pd.Timestamp(str(contract_year) + '-12-01')
            else:
                print("Invalid month code!")
                break
            expire_dates.append(expire_date)
            if df['trading_date'][i].year == contract_year:
                if df['trading_date'][i] > expire_date or df['trading_date'][i] == expire_date:
                    df.loc[i, 'tradeability'] = 'nontradeable'
                else:
                    df.loc[i, 'tradeability'] = 'tradeable'
            elif df['trading_date'][i].year != contract_year and df['trading_date'][i].month < expire_date.month:
                df.loc[i, 'tradeability'] = 'not to trade'
            else:
                df.loc[i, 'tradeability'] = 'tradeable'
        # INITIALIZE VARIABLES
        df['expire_date'] = expire_dates
        df['EMA20'] = ta.EMA(df['close'], timeperiod=20)
        df['EMA10'] = ta.EMA(df['close'], timeperiod=10)
        df['EMA5'] = ta.EMA(df['close'], timeperiod=5)
        df['ADX'] = ta.ADX(df['high'], df['low'], df['close'], timeperiod=14)
        df['BIAS'] = (df['close'] - df['close'].rolling(window=20).mean()) / df['close'].rolling(window=20).mean() * 100
        position = 0  
        initial_cash = 10000 # not exactly initial cash, but enough points to cover the margin (transform to monetary basis later)
        current_cash = initial_cash
        current_capital = initial_cash 
        trading_cost = 0  
        total_trading_cost = 0
        action = None
        trades = []
        daily = []
        for i in range(1, len(df)):
            trading_date = df['trading_date'][i]
            if df['tradeability'][i] == 'tradeable':
                # LONG TREND FOLLOWING STRATEGY
                if contract_strategy['LONG/SHORT'] == 'LONG':
                    if df['ADX'][i-1] > adx_optimal and \
                       df['close'][i-1] > df['EMA10'][i-1] and \
                       df['EMA10'][i-1] > df['EMA20'][i-1] and \
                       df['EMA10'][i-1] > df['EMA10'][i-2] and \
                       df['EMA20'][i-1] > df['EMA20'][i-2] and \
                       df['BIAS'][i-1] < bias_optimal and \
                       position < 3:
                        action = 'BUY'
                        price = df['open'][i]
                        position += 1
                        trading_cost = 0.0005 * price 
                        total_trading_cost += trading_cost
                        current_cash -= (price + trading_cost)
                        current_capital = current_cash + (position * df['open'][i])
                        trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                    elif position > 0:
                        if df['close'][i-1] < df['EMA10'][i-1] or \
                           df['BIAS'][i-1] > bias_optimal:
                            action = 'SELL'
                            price = df['open'][i]
                            current_cash += price * position
                            trading_cost = 0.0005 * price * position
                            position = 0
                            total_trading_cost += trading_cost
                            current_cash -= trading_cost
                            current_capital = current_cash
                            trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                        else:
                            action = 'HOLD'
                            price = None
                            trading_cost = 0
                            if df['close'][i] > df['close'][i-1] or \
                               df['close'][i] == df['close'][i-1]:
                                current_cash += position * (df['close'][i] - df['close'][i-1])
                                current_capital += position  * (df['close'][i] - df['close'][i-1])
                            else:
                                current_cash -= position * (df['close'][i-1] - df['close'][i])
                                current_capital -= position * (df['close'][i-1] - df['close'][i])
                    else:
                        action = None
                        trading_cost = 0
                # SHORT TREND FOLLOWING STRATEGY
                if contract_strategy['LONG/SHORT'] == 'SHORT':
                    if df['ADX'][i-1] > adx_optimal and \
                       df['close'][i-1] < df['EMA10'][i-1] and \
                       df['EMA10'][i-1] < df['EMA20'][i-1] and \
                       df['EMA10'][i-1] < df['EMA10'][i-2] and \
                       df['EMA20'][i-1] < df['EMA20'][i-2] and \
                       df['BIAS'][i-1] > bias_optimal and \
                       position < 1:
                        action = 'SELL'
                        price = df['open'][i]
                        position += 1
                        trading_cost = 0.0005 * price 
                        total_trading_cost += trading_cost
                        current_cash -= (price + trading_cost)
                        current_capital = current_cash + (position * df['open'][i])
                        trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                    elif position > 0:
                        if df['close'][i-1] > df['EMA10'][i-1] or \
                           df['BIAS'][i-1] < bias_optimal:
                            action = 'BUY'
                            price = df['open'][i]
                            current_cash += price * position
                            trading_cost = 0.0005 * price * position
                            position = 0
                            total_trading_cost += trading_cost
                            current_cash -= trading_cost
                            current_capital = current_cash
                            trades.append([trading_date, action, price, position, trading_cost, current_cash, current_capital]) 
                        else:
                            action = 'HOLD'
                            price = None
                            trading_cost = 0
                            if df['close'][i] < df['close'][i-1] or \
                               df['close'][i] == df['close'][i-1]:
                                current_cash += position * (df['close'][i-1] - df['close'][i])
                                current_capital += position * (df['close'][i-1] - df['close'][i])
                            else:
                                current_cash -= position * (df['close'][i] - df['close'][i-1])
                                current_capital -= position * (df['close'][i] - df['close'][i-1])
                    else:
                        action = None
                        trading_cost = 0
            #else:
                #action = None
                #trading_cost = 0
                daily_capital = current_cash + (position * df['close'][i])
                daily.append([trading_date, df['close'][i], action, df['open'][i], position, trading_cost, current_cash, current_capital, daily_capital]) 
            #check indent position!!!
            trades_df = pd.DataFrame(trades, columns=['trading_date', 'action', 'price', 'current_share_holding', 'trading_cost', 'current_cash', 'current_capital'])
            daily_df = pd.DataFrame(daily, columns=['date', 'close', 'action', 'open', 'current_share_holding', 'trading_cost', 'current_cash', 'current_capital', 'daily_capital'])
        
        # PERFORMANCE METRICS PER TRADE AND PERFORMANCE METRICS DAILY
        # TRANSFORM TO MONETARY BASIS
        daily_df['daily_returns_diff'] = daily_df['daily_capital'] - daily_df['daily_capital'].shift(1)
        daily_df['daily_point'] = daily_df['current_capital'] - 10000 
        daily_point = daily_df['daily_point']
        monetary_daily_profit = daily_point * multiplier
        daily_df['monetary_daily_profit'] = monetary_daily_profit
        
        trades_df['per_trade_returns_pct_change'] = trades_df['current_capital'].pct_change()
        daily_df['daily_returns_pct_change'] = daily_df['daily_capital'].pct_change()
        sharpe_ratio_per_trade = trades_df['per_trade_returns_pct_change'].mean() / trades_df['per_trade_returns_pct_change'].std()
        sharpe_ratio_daily = daily_df['daily_returns_pct_change'].mean() / daily_df['daily_returns_pct_change'].std()

        max_drawdown_per_trade = (1 - (trades_df['current_capital'] / trades_df['current_capital'].cummax())).max()
        max_drawdown_daily = (1 - (daily_df['daily_capital'] / daily_df['daily_capital'].cummax())).max()

        per_trade_returns_pct_change = trades_df['current_capital'].pct_change()
        def calculate_gain_to_pain_ratio_per_trade(per_trade_returns_pct_change):
            total_pain_per_trade = sum([r if r < 0 else 0 for r in per_trade_returns_pct_change])
            total_gain_per_trade = sum([r if r > 0 else 0 for r in per_trade_returns_pct_change])
            if total_pain_per_trade == 0:
                return float('inf')  
            else:
                return abs(total_gain_per_trade / total_pain_per_trade)
        gain_to_pain_ratio_per_trade = calculate_gain_to_pain_ratio_per_trade(per_trade_returns_pct_change)
       
        daily_returns_pct_change = daily_df['daily_capital'].pct_change()
        def calculate_gain_to_pain_ratio_daily(daily_returns_pct_change): 
            total_pain_daily = sum([r if r < 0 else 0 for r in daily_returns_pct_change])
            total_gain_daily = sum([r if r > 0 else 0 for r in daily_returns_pct_change])
            if total_pain_daily == 0:
                return float('inf') 
            else:
                return abs(total_gain_daily / total_pain_daily)
        gain_to_pain_ratio_daily = calculate_gain_to_pain_ratio_daily(daily_returns_pct_change)

        final_profit = (daily_df['daily_capital'].iloc[-1] - initial_cash) / df['close'].mean()
        trades_df['sharpe_ratio_per_trade'] = sharpe_ratio_per_trade
        trades_df['max_drawdown_per_trade'] = max_drawdown_per_trade
        trades_df['gain_to_pain_ratio_per_trade'] = gain_to_pain_ratio_per_trade
        
        daily_df['sharpe_ratio_daily'] = sharpe_ratio_daily
        daily_df['max_drawdown_daily'] = max_drawdown_daily
        daily_df['gain_to_pain_ratio_daily'] = gain_to_pain_ratio_daily
        daily_df['final_profit'] = final_profit
        
        # OUTPUT TRADE RECORDS AND DAILY PNLS ALONG WITH PERFORMANCE METRICS to "_trades.xlsx" and "_daily.xlsx"
        result_folder = "CTA_result"
        if not os.path.exists(result_folder):
            os.makedirs(result_folder)
        # OUTPUT FILE PATHS WITH SUFFIX DENOTING LONG OR SHORT STRATEGY
        output_suffix = '_long' if contract_strategy['LONG/SHORT'] == 'LONG' else '_short'
        trades_excel_path = os.path.join(result_folder, f"{os.path.basename(file_path).replace('.csv', '')}{output_suffix}_trades.xlsx")
        daily_excel_path = os.path.join(result_folder, f"{os.path.basename(file_path).replace('.csv', '')}{output_suffix}_daily.xlsx")
        trades_df.to_excel(trades_excel_path, index=False, float_format='%.4f')
        daily_df.to_excel(daily_excel_path, index=False, float_format='%.4f')
        
        print(f"Processed {file_path} and saved trade record to {trades_excel_path}")
        print("Sharpe Ratio Per Trade:", sharpe_ratio_per_trade)
        print("Max Drawdown Per Trade:", max_drawdown_per_trade)
        print("Gain-to-Pain Ratio Per Trade:", gain_to_pain_ratio_per_trade)
        
        print(f"Processed {file_path} and saved daily record to {daily_excel_path}")
        print("Sharpe Ratio Daily:", sharpe_ratio_daily)
        print("Max Drawdown Daily:", max_drawdown_daily)
        print("Gain-to-Pain Ratio Daily:", gain_to_pain_ratio_daily)
        print("Final Profit:", final_profit)
        
        # AGGREGATE PROFITS AND LOSSES FROM ALL CONTRACTS FOR TOTAL DAILY PNL
        aggregated_profits = {}
        # ITERATE OVER ALL FILES IN THE CTA_result FOLDER
        for filename in os.listdir(result_folder):
            if filename.endswith("_daily.xlsx"): 
                file_path = os.path.join(result_folder, filename)
                contract_code = filename.split("_")[0] 
                daily_df = pd.read_excel(file_path, header=None, skiprows=1)
                dates = daily_df.iloc[:, 0] 
                profits = daily_df.iloc[:, 11] # extract monetary daily profit
                actions = daily_df.iloc[:, 2]
                # ITERATE OVER EACH DATE, ACTION, AND PROFIT
                for date, action, profit in zip(dates, actions, profits):
                    if pd.notna(date):
                        if date in aggregated_profits:
                            existing_profit, existing_actions, contract_codes = aggregated_profits[date]
                        else:
                            existing_profit = 0
                            existing_actions = []
                            contract_codes = []
                        # SUM PROFIT FOR EACH DATE
                        new_profit = existing_profit + profit
                        existing_actions.append(action)
                        if action in ['BUY', 'SELL', 'HOLD']:
                            contract_codes.append(contract_code)
                        aggregated_profits[date] = (new_profit, existing_actions, contract_codes) # Update the aggregated_profits dictionary     
        aggregated_df = pd.DataFrame(list(aggregated_profits.items()), columns=['Date', 'Total Profit']) # Convert aggregated_profits dictionary to DataFrame
        # EXTRACT NUMERIC VALUE FROM 'Total Profit' USING REGULAR EXPRESSIONS
        def extract_numeric_value(text):
            matches = re.findall(r"[-+]?\d*\.\d+|\d+", str(text))  # Match decimal numbers
            if matches:
                return float(matches[0])  # Return the first match as float
            else:
                return None
        aggregated_df['Value'] = aggregated_df['Total Profit'].apply(extract_numeric_value)
        # SORT THE DATAFRAME BY DATE
        aggregated_df['Date'] = pd.to_datetime(aggregated_df['Date'], format='%Y-%m-%d') 
        aggregated_df = aggregated_df.dropna(subset=['Date'])  # Drop rows with invalid dates
        csv_file_path = os.path.join(result_folder, "monetary_daily_profit.csv")
        aggregated_df.to_csv(csv_file_path, index=False)
        print("Aggregated profits saved to:", csv_file_path)

if __name__ == "__main__":
    input_folder = "C:/Users/meng/Desktop/quant/CTA_data"
    result_folder = "C:/Users/meng/Desktop/quant/CTA_result"
    strategy_file = "C:/Users/meng/Desktop/quant/CTA_data/instruction/CTA_strategy.csv"
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)
    strategy_df = read_strategy(strategy_file)
    for filename in os.listdir(input_folder):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_folder, filename)
            process_csv(file_path, strategy_df)